# Validation of NER

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%env HOME=/content/drive/MyDrive/
#!mkdir -p ~/Research/huggingface
%cd ~/Research/ner/

env: HOME=/content/drive/MyDrive/
/content/drive/MyDrive/Research/ner


In [ ]:
#%%writefile ner_debugging.py
import json

def main():
    x=open('DSTC9_task1_log_eval_phon_templates.json','r')
    y=open('/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task2/data/val/labels.json','r')
    z=open('/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task2/data/knowledge.json','r')
    log=json.load(x)
    label=json.load(y)
    kb=json.load(z)

    i=0#keep track of which element in log/label (dialogue number)
    n=0#number of edge cases
    edge_cases=[]

    for dialogue in label:
        if dialogue['target']==True:
            flag=False
            log_dialogue=log[i]

            target_id=str(dialogue['knowledge'][0]['entity_id'])
            target_cat=dialogue['knowledge'][0]['domain']
            target='<'+target_cat+'-'+target_id+'>'

            res=[]
            for turn in log_dialogue:
                rturn={}
                rturn['text']=turn['text']
                rturn['template']=turn['template']
                res.append(rturn)
                for entity_id in turn['id_map']:
                    cat,id=entity_id.split('-')
                    if '|' in id:
                        ids=id.split('|')
                        for pos in ids:
                            new_id='<'+cat+'-'+pos+'>'
                            if new_id==target:
                                print(new_id,'==',target)
                                flag=True
                    else:
                        if entity_id==target:
                            flag=True
            if not flag:
                case={}
                case['case_id']='#'+str(n)
                case['entity_name']=kb[target_cat][target_id]['name']
                case['entity_id']='-'.join([target_cat,target_id])
                case['dialogue']=res
                edge_cases.append(case)
                print('Problem #',n,'-',i,'\t',case['entity_name'],' ',case['entity_id'])
                n+=1
        i+=1

    out=open('DSTC10_track2_task2_log_edgecases.json','w')
    json.dump(edge_cases,out,indent=4)

if __name__=='__main__':
    main()

Problem # 0 - 0 	 Grant Hotel   hotel-110053
Problem # 1 - 2 	 Underdogs Too   restaurant-120496
Problem # 2 - 10 	 Sutro's at The Cliff House   restaurant-120412
Problem # 3 - 11 	 Smokin' Warehouse Barbecue - Sacramento Street   restaurant-120384
Problem # 4 - 13 	 Nineteen 06 Mission   hotel-110134
Problem # 5 - 16 	 Boudin Bakery & Cafe   restaurant-120053
Problem # 6 - 17 	 Warakubune Sushi   restaurant-120510
Problem # 7 - 18 	 Franciscan Crab Restaurant   restaurant-120130
Problem # 8 - 19 	 Pier Market Seafood Restaurant   restaurant-120325
Problem # 9 - 20 	 Hotel Sunrise   hotel-110090
Problem # 10 - 23 	 San Francisco Zoo   attraction-100150
Problem # 11 - 24 	 San Francisco Museum of Modern Art   attraction-100147
Problem # 12 - 25 	 The Flying Falafel   restaurant-120448
Problem # 13 - 27 	 Twin Peaks   attraction-100187
Problem # 14 - 29 	 Inn San Francisco   hotel-110111
Problem # 15 - 30 	 Swan Oyster Depot   restaurant-120413
Problem # 16 - 36 	 Foreign Cinema   restau

# Text Classification Data Collection
316453 true vs. 220010 false


1.   split into 9:1 == 482815:53646




In [ ]:
%env HOME=/content/drive/MyDrive/
#!mkdir -p ~/Research/huggingface
%cd ~/Research/ner/text_classifier/

env: HOME=/content/drive/MyDrive/
/content/drive/MyDrive/Research/ner/text_classifier


In [ ]:
import json
def main():
    f=open('classifier_examples.json','r')
    f1=open('classifier_examples_train.json','w')
    f2=open('classifier_examples_test.json','w')
    dic1=[]
    n=53646
    dic=json.load(f)
    dic1=dic[:53646]
    dic2=dic[53646:]
    print('Dumping......')
    json.dump(dic1,f2,indent=4)
    json.dump(dic2,f1,indent=4)
    print('Done.')
  
if __name__=="__main__":
    main()

Dumping......
Done.


In [ ]:
import json
def main():
    f=open('classifier_examples.json','r')
    f1=open('classifier_examples_train.json','r')
    f2=open('classifier_examples_test.json','r')
    dic1=json.load(f1)
    dic2=json.load(f2)
    for elem in dic1:
        x=elem['body']
        print(x)
    for elem in dic2:
        x=elem['body']
        print(x)
  
if __name__=="__main__":
    main()

In [ ]:
import torch

model_name = "bert-base-uncased" # pretrained model name
batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
dataroot = '/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data/knowledge.json'
dataroot_eval = '/content/drive/MyDrive/Research/ner/text_classifier'

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModel

config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Set device
model.to(device)

In [ ]:
#@title Knowledge DB
#%%writefile knowledge_db.py
import re
import os
import json
import string
from torch.utils.data import Dataset

import nltk
from nltk.stem.snowball import SnowballStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords

class KnowledgeDataset(Dataset):
  """
  Yield delexicalized title utterances
  """
  def __init__(self, dataroot, filename, tokenizer, title_field='title_ex', body_field='body_ex'):
    path = os.path.join(os.path.abspath(dataroot))
    
    kb_file = os.path.join(path, filename)
    with open(kb_file, 'r') as f:
      self.kb = json.load(f)

    self.ps = SnowballStemmer("english", ignore_stopwords=True)
    self.stopwords = stopwords.words('english')

    self.kid2doc = {}
    self.kb_utterances = {}
    self.kb_utterances_list = []

    # remove stopwords, sort remaining keywords on title
    self.kb_utterances1 = {}
    self.kb_utterances_list1 = []

    # remove stopwords, sort remaining keywords on body
    self.kb_utterances2 = {}
    self.kb_utterances_list2 = []

    self.entity_utterances = {}

    for domain in self.kb.keys():
      self.entity_utterances[domain] = []
      for i, entity in self.kb[domain].items():
        name = entity["name"]
        titles = []
        for j, qa, in entity["docs"].items():
          kid = "{}_{}_{}".format(domain, i, j)
          # Use title_template first. If not found, use the default title
          title = qa[title_field] if title_field in qa else qa["title_template"]
          body = qa[body_field] if body_field in qa else qa["body_template"]

          self.kid2doc[kid] = title + "\t" + body

          q = self.delexicalize(domain, name, title)
          q1 = self.norm_query(q)

          a = self.delexicalize(domain, name, body)
          a1 = self.norm_query(a)

          titles.append(q)

          if q not in self.kb_utterances:
            self.kb_utterances[q] = [kid]
            # only store uniq query
            self.kb_utterances_list.append(q)
          else:
            self.kb_utterances[q].append(kid)
            
          if q1 not in self.kb_utterances1:
            self.kb_utterances1[q1] = [kid]
            # only store uniq query
            self.kb_utterances_list1.append(q1)
          else:
            self.kb_utterances1[q1].append(kid)

          if a1 not in self.kb_utterances2:
            self.kb_utterances2[a1] = [kid]
            # only store uniq query
            self.kb_utterances_list2.append(a1)
          else:
            self.kb_utterances2[a1].append(kid)

        self.entity_utterances[domain].append(titles)
    self.tokenizer = tokenizer

    # merge equivalent entity docs in kb_utterances1 and kb_utterances2 
    self.doc2clusterIdx = {}
    self.doc_clusters = []
    num_cluster = 0

    # get the doc subsets
    for S in self.kb_utterances1.values():
      # check if any docid exists in current map
      c = None
      for docid in S:
        if docid in self.doc2clusterIdx:
          c = self.doc2clusterIdx[docid]
          break
      if c is None:
        # create new cluster idx c
        c = num_cluster
        num_cluster += 1
        self.doc_clusters.append(set({}))
      # add all docid into c
      for docid in S:
        self.doc2clusterIdx[docid] = c
        self.doc_clusters[c].add(docid)
        
    self.doc2clusterIdx2 = {}
    self.doc_clusters2 = []
    num_cluster2 = 0

    # get the doc subsets
    for S in self.kb_utterances2.values():
      # check if any docid exists in current map
      c = None
      for docid in S:
        if docid in self.doc2clusterIdx2:
          c = self.doc2clusterIdx2[docid]
          break
      if c is None:
        # create new cluster idx c
        c = num_cluster2
        num_cluster2 += 1
        self.doc_clusters2.append(set({}))
      # add all docid into c
      for docid in S:
        self.doc2clusterIdx2[docid] = c
        self.doc_clusters2[c].add(docid)

    # merge set 2
    for S in self.kb_utterances2.values():
      # check if any docid exists in current map
      cluster_set = {}
      for docid in S:
        if docid in self.doc2clusterIdx:
          c = self.doc2clusterIdx[docid]
          # keep count. We only trust count > 1 (multiple docids from body map to the same title cluster)
          if c in cluster_set:
            cluster_set[c] += 1
          else:
            cluster_set[c] = 1

      # now we should merge all docs in cluster_set into a new cluster index
      new_cluster = set({})
      for i in cluster_set.keys():
        # only trust merging clusters with count > 1
        if cluster_set[i] <= 1:
          continue
        for docid in self.doc_clusters[i]:
          # update docid to the new cluster idx
          self.doc2clusterIdx[docid] = num_cluster
          new_cluster.add(docid)
        # clear i
        self.doc_clusters[i] = set({})

      # discard the body cluster due to potential noise
      #for docid in S:
      #  self.doc2clusterIdx[docid] = num_cluster
      #  new_cluster.add(docid)

      self.doc_clusters.append(new_cluster)
      num_cluster += 1

    # remove singleton clusters
    self.doc_clusters = [c for c in self.doc_clusters if len(c) > 0]
    self.doc_clusters2 = [c for c in self.doc_clusters2 if len(c) > 0]

  def get_doc(self, kid):
    try:
      return self.kid2doc[kid]
    except:
      return "Not found"

  def norm_query(self, query):
    """
    remove stopwords, stem each word, sort the word order, remove punctuation
    """
    query = query.lower()
    # remove anything after
    #query = re.sub(r"this \w+", "", query)
    #query = re.sub(r"this restaurant", "", query)
    #query = re.sub(r"this attraction", "", query)
    #query = re.sub(r"this facility", "", query)
    #query = re.sub(r"this property", "", query)
    #query = re.sub(r"this location", "", query)
    #query = re.sub(r"this site", "", query)
    #query = re.sub(r"this place", "", query)
    #query = re.sub(r"this spot", "", query)
    tokens = query.split()
    tokens = [self.ps.stem(w) for w in tokens if w not in self.stopwords]
    tokens = [w.translate(str.maketrans('', '', string.punctuation)) for w in tokens]
    tokens = [self.ps.stem(w) for w in tokens if w not in self.stopwords]
    return " ".join(sorted(tokens))
  
  def delexicalize(self, domain, name, query):
    """
    Replace occurrence of entity in sentence by domain
    For example: domain = 'hotel', name = 'A and B Guest House'
    Can I bring my pet to A and B Guest House? -> Can I bring my pet to this hotel?
    """
    # TODO: Need Trie here for fast entity matching
    return query

  def __iter__(self):
    for q in self.kb_utterances_list:
      yield q

  def __getitem__(self, index):
    return self.kb_utterances_list[index]

  def __len__(self):
    return len(self.kb_utterances_list)

  def save(self, out_file):
    with open(out_file, 'w') as f:
      for q in self.kb_utterances_list:
        f.write("{}\n".format(q))

  def collate_fn(self, batch):
    """
    Received a batch of queries.
    Return: dictionary of inputs
    """
    output = self.tokenizer(batch, padding=True, add_special_tokens = True, return_tensors="pt")
    output_dict = {x: output[x].to(device) for x in output}
    output_dict["query"] = batch

    return output_dict

[nltk_data] Downloading package stopwords to
[nltk_data]     /content/drive/MyDrive/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from torch.utils.data import DataLoader, SequentialSampler

# Use Seeger's version of knowledge.json with delexicalized sentence templates
#kb_dataset = KnowledgeDataset(dataroot, 'knowledge.json', tokenizer)
kb_eval_dataset = KnowledgeDataset(dataroot_eval, 'kb_eval_ex.json', tokenizer, title_field="title_ex")

kb_eval_dataloader = DataLoader(
  kb_eval_dataset,
  sampler=SequentialSampler(kb_eval_dataset),
  batch_size=batch_size,
  collate_fn=kb_eval_dataset.collate_fn
)

In [ ]:
#quick info debugging
print(len(kb_eval_dataset.kb_utterances))
print(len(kb_eval_dataset.kb_utterances1))
print(len(kb_eval_dataset.kb_utterances2))
print(kb_eval_dataset.kb_utterances1)
print(kb_eval_dataset.kb_utterances2)

print(len(kb_eval_dataset.doc_clusters))
print(kb_eval_dataset.doc_clusters)

5562
3350
3746
{'children welcom': ['hotel_0_0', 'hotel_8_8', 'hotel_23_4', 'hotel_29_17'], 'bring pet': ['hotel_0_1', 'hotel_8_4', 'hotel_11_5', 'hotel_27_5', 'hotel_32_20', 'hotel_110026_4', 'hotel_110124_0', 'hotel_110193_1', 'hotel_110202_3', 'restaurant_120062_1', 'train_*_18', 'attraction_100176_0'], 'guest onsit park': ['hotel_0_2'], 'checkin time': ['hotel_0_3', 'hotel_5_11', 'hotel_9_32', 'hotel_12_11', 'hotel_13_2', 'hotel_14_18', 'hotel_15_7', 'hotel_19_23', 'hotel_21_20', 'hotel_23_10', 'hotel_27_2', 'hotel_32_2', 'hotel_110002_17', 'hotel_110005_18', 'hotel_110006_21', 'hotel_110009_19', 'hotel_110015_14', 'hotel_110018_22', 'hotel_110019_14', 'hotel_110020_15', 'hotel_110021_17', 'hotel_110024_17', 'hotel_110028_18', 'hotel_110043_18', 'hotel_110045_21', 'hotel_110046_15', 'hotel_110050_22', 'hotel_110053_14', 'hotel_110089_20', 'hotel_110090_15', 'hotel_110094_20', 'hotel_110102_20', 'hotel_110103_16', 'hotel_110118_19', 'hotel_110121_13', 'hotel_110140_19', 'hotel_11014

In [ ]:
#look at content of the clusters
X = [C for C in kb_eval_dataset.doc_clusters if len(C) > 0]
Y = [C for C in kb_eval_dataset.doc_clusters2 if len(C) > 0]

print(len(X))
print(len(Y))
'''
for i in range(len(X)):
  for kid in X[i]:
    print(i, kid, kb_eval_dataset.get_doc(kid))
  print("\n\n")

for i in range(len(Y)):
  for kid in Y[i]:
    print(i, kid, kb_eval_dataset.get_doc(kid))
  print("\n\n")
'''

2939
3746


'\nfor i in range(len(X)):\n  for kid in X[i]:\n    print(i, kid, kb_eval_dataset.get_doc(kid))\n  print("\n\n")\n\nfor i in range(len(Y)):\n  for kid in Y[i]:\n    print(i, kid, kb_eval_dataset.get_doc(kid))\n  print("\n\n")\n'

In [ ]:
#produce pickle objects
import pickle

with open("kb_doc_clusters.pkl", "wb") as F:
  pickle.dump(kb_eval_dataset.doc_clusters, F)

with open("kb_doc_clusters2.pkl", "wb") as F:
  pickle.dump(kb_eval_dataset.doc_clusters2, F)

In [ ]:
#%%writefile example_generation_methods.py
import random
import json
import re
import pickle
import time

def postprocess_text(query):
    #assumes sen to be delexicalized
    #remove from delexicalized data the likes of 'at this hotel'
    #only concerned about intent, delete most of the slot, domain references
    #will remove 'at this time'
    delete_list=[r" at this \w+",r" at the \w+",r" to the \w+",r" to this \w+",r" on this \w+",r" at your \w+",r" to your \w+",r" on your \w+",r" on the \w+"]
    #[r" on the \w+"]
    #figure out how to remove things like on this restaurant's menu?
    repl_list=[r"this \w+",r"This \w+"]

    for elem in delete_list:
        query = re.sub(elem, "", query)
    for elem in repl_list:
        query = re.sub(elem, "it", query)
        if query[0:2]=='it':#capitalize
            query=query.replace('it','It',1)
    #query = re.sub(r"This restaurant restaurant", "It", query)
    #query = re.sub(r"at this \w+", "", query)
    #query = re.sub(r"at the \w+", "", query)
    #query = re.sub(r"This \w+", "It ", query)

    return query

def special_mod(title,body):
    title_ex=title
    body_ex=body
    title=postprocess_text(title)
    body=postprocess_text(body)
    delete_list=[" onboard"," on board"," at any train station"," for all the trains"," for a delayed or cancelled train", r" in your \w+",r" onto the \w+", r" on the \w+",r" at the \w+",
                 r" in the \w+",r" for the \w+",r" to the \w+", " to get"," station"]
    for elem in delete_list:
        title_ex=re.sub(elem,"",title_ex)
        body_ex=re.sub(elem,"",body_ex)
    title_ex=title_ex.replace("Are the trains","Are they")
    body_ex=body_ex.replace("Are the trains","Are they")
    title_ex=title_ex.replace("all the trains","they are")
    body_ex=body_ex.replace("all the trains","they are")
    print(title_ex,body_ex,sep='\n')
    return title_ex,body_ex


def modify_kb():
    kb=open('/content/drive/MyDrive/Research/ner/DSTC9_task1_kb_eval_phon_templates.json','r')
    dic=json.load(kb)
    kb.close()
    new=open('kb_eval_ex.json','w')
    n=0
    for cat in dic:
        for elem in dic[cat]:
            if dic[cat][elem]['name'] is None:#train or taxi
                for entry in dic[cat][elem]['docs']:
                    curr=dic[cat][elem]['docs'][entry]
                    #print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                    title_ex,body_ex=special_mod(curr['title'],curr['body'])
                    curr['title_ex']=title_ex
                    curr['body_ex']=body_ex
                continue
            for entry in dic[cat][elem]['docs']:
                curr=dic[cat][elem]['docs'][entry]
                title=postprocess_text(curr['title_template'])
                body=postprocess_text(curr['body_template'])
                #print(title)
                #print(body)
                curr['title_ex']=title
                curr['body_ex']=body
                n+=1
                print('line #',n,'--------')
    json.dump(dic,new,indent=4)
    print('Done.')

def get_cluster_map():
    res={}
    with open('kb_doc_clusters.pkl','rb') as F:
        pickle_obj1=pickle.load(F)
    with open('kb_doc_clusters2.pkl','rb') as F:
        pickle_obj2=pickle.load(F)
    lst1=[elem for elem in pickle_obj1 if len(elem)>1]#title matches
    lst2=[elem for elem in pickle_obj2 if len(elem)>1]#body matches

    for i in range(len(lst1)):
        res[i]=lst1[i]
    for j in range(len(lst2)):
        res[j]=lst2[j]
    return res
    

def more_neg(dic,cl_map,out,hm):
    n=0#example of negatives formed, 219918
    for cat in dic:
        for elem in dic[cat]:
            entity=dic[cat][elem]['name']
            doc_lst=dic[cat][elem]['docs']
            for entry in doc_lst:
                doc_id='_'.join([cat,elem,entry])
                curr=doc_lst[entry]
                for match in doc_lst:
                    mlst=[]
                    match_id='_'.join([cat,elem,match])
                    if match!=entry:
                        nv_flag=False#not valid flag
                        for cluster_id in cl_map:
                            if doc_id in cl_map[cluster_id]:
                                if match_id in cl_map[cluster_id]:
                                    nv_flag=True
                        if not nv_flag:
                            candidate={}
                            candidate['title']=doc_lst[entry]['title_ex']
                            candidate['body']=doc_lst[match]['body_ex']
                            candidate['label']='false'
                            pair=(candidate['title'],candidate['body'])
                            if pair not in hm:
                                print('execution #',n)
                                print(pair)
                                out.append(candidate)
                                hm[pair]=n
                                n-=1

                        


def find_comb(tlst,blst):
    rlst=[]
    for title in tlst:
        for i in range(len(blst)):
            pair=(title,blst[i])
            rlst.append(pair)#can have dupes, treated later with hm
    return rlst

def more_positive_ex(lst,typ):
    kb=open('kb_eval_ex.json','r')
    dic=json.load(kb)
    kb.close()

    ex_lst=[]
    tlst=[]#title list
    blst=[]#body list
    for elem in lst:
        cat,num,doc_id=elem.split('_')
        title=dic[cat][num]['docs'][doc_id]['title_ex']
        body=dic[cat][num]['docs'][doc_id]['body_ex']
        tlst.append(title)
        blst.append(body)
        #get every possible title/body out
    if typ==1:
        comb=find_comb(tlst,blst)
    elif typ==2:
        comb=find_comb(blst,tlst)
    for elem in comb:#dupes not checked here
        candidate={}
        if typ==1:
            candidate['title']=elem[0]
            candidate['body']=elem[1]
        else:
            candidate['title']=elem[1]
            candidate['body']=elem[0]
        candidate['label']='true'
        
        ex_lst.append(candidate)
        continue
    return ex_lst

def generate_from_pkl(hm,res):
    n=0#number of matched positive examples, 305016
    with open('kb_doc_clusters.pkl','rb') as F:
        pickle_obj1=pickle.load(F)
    with open('kb_doc_clusters2.pkl','rb') as F:
        pickle_obj2=pickle.load(F)
    lst1=[elem for elem in pickle_obj1 if len(elem)>1]#title matches
    lst2=[elem for elem in pickle_obj2 if len(elem)>1]#body matches
    for pairs in lst1:
        examples=more_positive_ex(pairs,1)
        for ex in examples:
            pair=(ex['title'],ex['body'])
            if pair not in hm:
                print(pair[0])
                print(pair[1])
                hm[pair]=str(n)
                n+=1
                print('execution #',n,'----------')
                res.append(ex)
    for pairs in lst2:
        examples=more_positive_ex(pairs,2)
        for ex in examples:
            pair=(ex['title'],ex['body'])
            if pair not in hm:
                print(pair[0])
                print(pair[1])
                hm[pair]=str(n)
                n+=1
                print('execution #',n,'----------')
                res.append(ex)
    return res,hm


def generate_from_kb():
    kb=open('kb_eval_ex.json','r')
    res=[]
    hm={}
    dic=json.load(kb)
    kb.close()
    n=0#total number of generic positive examples, 12008 (10483??)
    for category in dic:
        for number in dic[category]:
            entity_name=dic[category][number]['name']
            for entry in dic[category][number]['docs']:
                curr=(entry,dic[category][number]['docs'][entry])
                pair=(curr[1]['title_ex'],curr[1]['body_ex'])
                candidate={}
                candidate['title']=curr[1]['title_ex']
                candidate['body']=curr[1]['body_ex']
                candidate['label']='true'
                if pair not in hm:
                    hm[pair]=str(n)
                    res.append(candidate)
                    n+=1
                    print('execution #',n)
    print('Done.')
    return hm,res

def count_pairs(filepath):
    kb=open(filepath,'r')
    dic=json.load(kb)
    kb.close()
    n=0
    for category in dic:
        for number in dic[category]:
            if dic[category][number]['name'] is None:
                continue
            for entry in dic[category][number]['docs']:
                n+=1
    print(n)

def main():
    #print(postprocess_text("Live music is not being offered at this time at this train station."))
    modify_kb()
    '''
    kb=open('kb_eval_ex.json','r')
    dic=json.load(kb)
    kb.close()

    examples=open('classifier_examples.json','w')

    print('Generic Positives ------------------------------------------')
    map,lst=generate_from_kb()
    time.sleep(10)
    print('More positives ------------------------------------------')
    lst,map=generate_from_pkl(map,lst)
    time.sleep(10)
    print('Negatives ------------------------------------------')
    cl_map=get_cluster_map()
    more_neg(dic,cl_map,lst,map)
    random.shuffle(lst)
    json.dump(lst,examples,indent=4)
    examples.close()
    '''
if __name__=='__main__':
    main()

Streaming output truncated to the last 5000 lines.
line # 7072 --------
line # 7073 --------
line # 7074 --------
line # 7075 --------
line # 7076 --------
line # 7077 --------
line # 7078 --------
line # 7079 --------
line # 7080 --------
line # 7081 --------
line # 7082 --------
line # 7083 --------
line # 7084 --------
line # 7085 --------
line # 7086 --------
line # 7087 --------
line # 7088 --------
line # 7089 --------
line # 7090 --------
line # 7091 --------
line # 7092 --------
line # 7093 --------
line # 7094 --------
line # 7095 --------
line # 7096 --------
line # 7097 --------
line # 7098 --------
line # 7099 --------
line # 7100 --------
line # 7101 --------
line # 7102 --------
line # 7103 --------
line # 7104 --------
line # 7105 --------
line # 7106 --------
line # 7107 --------
line # 7108 --------
line # 7109 --------
line # 7110 --------
line # 7111 --------
line # 7112 --------
line # 7113 --------
line # 7114 --------
line # 7115 --------
line # 7116 --------
line

# Token classification

In [ ]:
#To see GPU info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [ ]:
%env HOME=/content/drive/MyDrive/
#!mkdir -p ~/Research/huggingface
%cd ~/Research/ner
#%env PYTHONPATH=/content/drive/MyDrive/Research/huggingface/transformers/src/transformers:/content/drive/MyDrive/Research/huggingface/datasets

env: HOME=/content/drive/MyDrive/
/content/drive/MyDrive/Research/ner


In [ ]:
from simple_tokenize import Clean_Text

In [ ]:
#@title Simple Tokenize (NO CLEAN TEXT)
#%%writefile simple_tokenize.py
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
import string

def Word_Tokenize(text):
    tokens=word_tokenize(text)
    ss=text.split()
    i=0
    z=[]
    left=''
    for w in tokens:
        if i<len(ss) and w==ss[i]:
            z.append(w)
            i+=1
        elif w in string.punctuation:
            if left:
                z.append(left)
                left=""
                i+=1
            z.append(w)
        else:
            #glue w into left
            left+=w
            if left==ss[i]:
                z.append(left)
                left=''
                i+=1
    return z

s="\\ni didn't do this, because you're quite right!!. m I right?"
a='100,000 dollar!!.'
b="minutes.\nreference"
c="33-34 bucks"
print('for\t',b,Word_Tokenize(b))
print('for\t',c,Word_Tokenize(c))


[nltk_data] Downloading package punkt to
[nltk_data]     /content/drive/MyDrive/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
for	 minutes.
reference ['minutes', '.', 'reference']
for	 33-34 bucks ['33-34', 'bucks']


In [ ]:
#Trying lfs to git - failed
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install

In [ ]:
!git add 'phonemes_ver.zip' 

In [ ]:
!pwd

/content/drive/My Drive/Research/ner/Multiwoz/multiwoz/data/MultiWOZ_2.2/run_env


# Grapheme to Phoneme for DSTC10

In [ ]:
!pip install g2p_en

  Using cached https://files.pythonhosted.org/packages/d7/d9/b77dc634a7a0c0c97716ba97dd0a28cbfa6267c96f359c4f27ed71cbd284/g2p_en-2.1.0-py3-none-any.whl
Processing /content/drive/MyDrive/.cache/pip/wheels/d5/aa/e1/dbba9e7b6d397d645d0f12db1c66dbae9c5442b39b001db18e/Distance-0.1.3-cp37-none-any.whl


In [ ]:
!zip phonemes_ver.zip PHONEME_DSTC9_task1_train_log_templates.json PHONEME_DSTC9_task1_val_log_templates.json PHONEME_DSTC9_task1_kb_templates.json PHONEME_DSTC9_task1_log_eval_templates.json PHONEME_DSTC9_task1_kb_eval_templates.json PHONEME_DSTC10_task2_log_templates.json

In [ ]:
#@title G2P plain
#%%writefile g2p_methods.py
from g2p_en import G2p
import json

def g2p_log(logfile,outfile):
    g2p = G2p()
    out=open(outfile,'w')
    data=open(logfile,'r')
    logs=json.load(data)
    data.close()

    n=0
    for dialogue in logs:
        for section in dialogue:
            if section['text'] is not None:
                phoneme=g2p(section['text'])
                section['phoneme']=phoneme
            n+=1
            print('line/execution #',n)

    print('dumping...')
    json.dump(logs,out,indent=4)
    out.close()
    print('Done.')


def g2p_kb(kb_file,outfile):
    g2p = G2p()
    out=open(outfile,'w')
    kb=open(kb_file,'r')
    dic=json.load(kb)
    kb.close()

    n=0
    for cat in dic:
        for num in dic[cat]:
            entity=dic[cat][num]['name']
            if not entity:#skip the taxi stuff
                continue
            sen_dic=dic[cat][num]['docs']
            for entry in sen_dic:
                if sen_dic[entry]['title'] is not None:
                    title_phoneme=g2p(sen_dic[entry]['title'])
                    sen_dic[entry]['title_phoneme']=title_phoneme
                if sen_dic[entry]['title'] is not None:
                    body_phoneme=g2p(sen_dic[entry]['body'])
                    sen_dic[entry]['body_phoneme']=body_phoneme
                print('line/execution #',n)
                n+=1

    print('dumping...')
    json.dump(dic,out,indent=4)
    out.close()
    print('Done.')

def main():
    '''INFO
    OUTPUT -> REFERENCED INPUT

    DSTC9:
    'PHONEME_DSTC9_task1_train_log_templates.json' -> 'logs_mod.json' (train,768500)
    'PHONEME_DSTC9_task1_val_log_templates.json' -> '/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data/val/logs.json' (val,101649)
    'PHONEME_DSTC9_task1_kb_templates.json' -> 'knowledge_mod.json' (kb)

    DSTC10:
    'PHONEME_DSTC9_task1_log_eval_templates.json' -> '/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data_eval/test/logs.json' (test/eval from dstc9,36091)
    'PHONEME_DSTC9_task1_kb_eval_templates.json' -> '/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data_eval/knowledge.json' (kb_test/eval from dstc9)
    'PHONEME_DSTC10_task2_log_templates.json' -> '/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task2/data/val/logs.json' (val)
    '''

    log_outfiles=['PHONEME_DSTC9_task1_train_log_templates.json','PHONEME_DSTC9_task1_val_log_templates.json','PHONEME_DSTC9_task1_log_eval_templates.json','PHONEME_DSTC10_task2_log_templates.json']
    log_files=['logs_mod.json','/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data/val/logs.json','/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data_eval/test/logs.json',
               '/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task2/data/val/logs.json']
    for i in range(len(log_outfiles)):
        g2p_log(log_files[i],log_outfiles[i])
        print('DONE WITH LOG              #',i)
    print('DONE WITH LOGS--------------------------------------------------------------')

    kb_outfiles=['PHONEME_DSTC9_task1_kb_templates.json','PHONEME_DSTC9_task1_kb_eval_templates.json']
    kb_files=['knowledge_mod.json','/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data_eval/knowledge.json']
    for i in range(len(kb_outfiles)):
        g2p_kb(kb_files[i],kb_outfiles[i])
        print('DONE WITH KNOWLEDGE              #',i)
    print('DONE WITH KNOWLEDGE--------------------------------------------------------------')

if __name__=='__main__':
    main()

In [ ]:
from g2p_en import G2p
import nltk
import re

#nltk.download('punkt')
g2p=G2p()
text="reservation,?"
#print(re.findall(r"[\w']+|[.,!?;]", text))
res=g2p(text)
#res2=nltk.word_tokenize(text)
print(' '.join(res))
print(res2)

In [ ]:
from num2words import num2words
print(num2words('2000'))

two thousand


# Install huggingface stuff
Huggingface's transformers library, pretrained models, and datasets have very good support.

* `transformers` library
* `datasets` library. Also look at [Colab notebook](https://github.com/huggingface/datasets/blob/master/notebooks/Overview.ipynb) for details.

Besides, we need to install other library, including sequential evaluation metrics
* `seqeval` library

In [ ]:
# You are supposed to run just after you first run this notebook! We want to fix this library to ensure reproducibility.
#!git clone https://github.com/huggingface/transformers.git
!git clone https://github.com/huggingface/datasets.git

Cloning into 'datasets'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 28071 (delta 23), reused 35 (delta 12), pack-reused 28012
Receiving objects: 100% (28071/28071), 43.56 MiB | 7.07 MiB/s, done.
Resolving deltas: 100% (9882/9882), done.
Checking out files: 100% (4867/4867), done.


In [ ]:
#What's this for??
%cd ~/Research/huggingface/datasets/
!pip install -e .
import datasets
print(datasets.__path__)

In [ ]:
# You are supposed to run everytime you run this Colab notebook
!pip install seqeval

In [ ]:
# You are supposed to run everytime you run this Colab notebook
#%cd ~/Research/huggingface/transformers/
#!python setup.py install

!pip install transformers

In [ ]:
# You are supposed to run everytime you run this Colab notebook
'''
%cd ~/Research/huggingface/datasets/
!python setup.py install

!ls /usr/local/lib/python3.6/dist-packages | grep transformer
'''
!pip install datasets
#!pip uninstall datasets

#%cd ~/

In [ ]:
# You are supposed to run everytime you run this Colab notebook

# Make sure that we have a recent version of pyarrow in the session before we continue - otherwise reboot Colab to activate it
import pyarrow
if int(pyarrow.__version__.split('.')[1]) < 16 and int(pyarrow.__version__.split('.')[0]) == 0:
    import os
    os.kill(os.getpid(), 9)

In [ ]:
# sanity check
import transformers
print(transformers.__path__)
from transformers import BertForTokenClassification
from datasets import load_dataset, ClassLabel

['/usr/local/lib/python3.7/dist-packages/transformers']


## Run torch example
* See transformers/examples, there are many other task examples to copy from.\
* Refer to trasnformers/examples/README.md

In [ ]:
%cd ~/Research/huggingface/transformers/examples/token-classification
!bash run.sh

#Text2num Experiments

In [ ]:
%env HOME=/content/drive/MyDrive/
%cd ~/Research/ner/text2num
!ls -lrt

env: HOME=/content/drive/MyDrive/
/content/drive/MyDrive/Research/ner/text2num
total 10147
-rw------- 1 root root 4593300 Aug 26 15:10 new_logs.json
-rw------- 1 root root 5784270 Aug 26 15:27 converted_logs.json
-rw------- 1 root root    7426 Aug 26 15:30 extra_methods.py
drwx------ 2 root root    4096 Aug 26 15:33 __pycache__


In [ ]:
#Test
convert_text2num('first one')

'2'

In [ ]:
import json
#from extra_methods import convert_text2num

f1 = open('new_logs.json','r')
log = json.load(f1)

for dialogue in log:
    for turn in dialogue:
        temp = convert_text2num(turn['text']).strip()
        turn['converted_text'] = temp
        print(temp)

out = open('converted_logs.json','w')
json.dump(log, out, indent = 4)
out.close()

hi hey i'm planning a trip to san francisco and i'm wondering if you cald help me u umm ok a couple request the first one is a place to eat umm in the inner richmond
sure i'm finding quite a few locations in inner richmond but i know of a great spot it's called cinderella bakery and cafe
what kind of price range is that in
sure they are in a moderate price range
ok and do they have a happy hour
let me check. they do not have a happy hour
ok so i sould be able to doin any time uh what's the address of the restaurant
yeah for sure they are located at 436 balboa street
perfect then ummm after i go there is there any chance there's kind of a museum or a place to go around there
definitely i really recommend the exploratorium after dark
oh that sounds fun uh what's the address there
they're located at pier 	0	15
they're located at pier 15
od ok you kno i need an appointment to visit them do we let me hape to make a reservation
sure let me check on that it looks like. they are not by appoint

# Task 1: Run on DSTC9 data(Includes DSTC10 preprocessing)

* Goal: Extract named entities such as hotel names, restaurant names that are defined in a provided knowledge base file

In [ ]:
!git clone https://github.com/alexa/alexa-with-dstc9-track1-dataset.git

In [ ]:
!git clone https://github.com/alexa/alexa-with-dstc10-track2-dataset.git

In [ ]:
%env HOME=/content/drive/MyDrive/
%cd ~/Research/ner/

env: HOME=/content/drive/MyDrive/
/content/drive/MyDrive/Research/ner


In [ ]:
#for secondary processing under DSTC10 NER and g2p conversion
!pip install fastDamerauLevenshtein
!pip install g2p_en
!pip install num2words

## Testing functions:

In [ ]:
import re

sen="Does Rosa\u2019s Bed and Breakfast have a swimming pool?"
re.sub(r'\\\\u2019',"'",sen)

'Does Rosa’s Bed and Breakfast have a swimming pool?'

In [ ]:
#Test dl function
from fastDamerauLevenshtein import damerauLevenshtein as dl

word1='soo'
word2='zoo'

dl_score=dl(word1,word2,similarity=False,deleteWeight=1,insertWeight=2,replaceWeight=2)
print(dl_score)

1.0


In [ ]:
from num2words import num2words
print(num2words('06'))

six


In [ ]:
from extra_methods import convert_text2num
convert_text2num('nineteen zero six mission')

'1906 mission '

In [ ]:
convert_text2num('')

'second three '

In [ ]:
from simple_tokenize import Clean_Text
from simple_tokenize import Word_Tokenize

text="Super Duper Burger - Washington DC's in the\n\ngame"
print(Clean_Text(text))
print(Word_Tokenize(Clean_Text(text)))

Super Duper Burger - Washington DC's in the

game
['Super', 'Duper', 'Burger', '-', 'Washington', 'DC', "'s", 'in', 'the', 'game']


# Process with Clean Text

In [ ]:
!git clone https://github.com/alexa/alexa-with-dstc10-track2-dataset.git

Cloning into 'alexa-with-dstc10-track2-dataset'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 166 (delta 75), reused 86 (delta 29), pack-reused 0
Receiving objects: 100% (166/166), 12.35 MiB | 5.99 MiB/s, done.
Resolving deltas: 100% (75/75), done.
Checking out files: 100% (45/45), done.


In [ ]:
%cd DSTC10_evaluation

REMEMBER. HAVE TO RERUN THIS ON OG KB EXCEPT FOR KB_EVAL!!!

In [ ]:
#@title Process_clean
%%writefile Process_clean_for_eval.py
import time
import json
import re
import argparse
from BabyTrie_DSTC10_v5 import BabyTrie
from extra_methods import convert_text2num#may have bugs such as for 'second one' = 3
from g2p_en import G2p
from simple_tokenize import Clean_Text
from simple_tokenize import Word_Tokenize#the words are tokenized the same way; can choose not to use this

def getphoneme(g2p,text):
    ptext=g2p(text)
    string='-'.join(ptext)
    res=string.replace('- -','_')
    return res

def gettemplate_wmap(bt,g2p,title):
    title = re.sub(r'\u2019', "'", title)
    title=Clean_Text(title).replace('\n',' ')
    nltk=' '.join(Word_Tokenize(title))

    word_lst,ind_dic=bt.isinTrie(title)
    nlst=Word_Tokenize(Clean_Text(title))
    new_sen=title.strip()#cleans up sentence
    
    en_dic={}
    title_phoneme=title
    for key in ind_dic:
        ind_lst=ind_dic[key]
        for pair in ind_lst:
            temp_str=''
            for index in range(pair[0][0],pair[0][1]+1):
                if not nlst[index][0].isalnum():
                    temp_str=temp_str[:-1]
                if nlst[index]=='-':
                    temp_str+=nlst[index]
                else:
                    temp_str+=nlst[index]+' '
            temp_str=temp_str[:-1]#getting rid of last space
            #temp_str_converted=convert_text2num(temp_str)#name entity to be in dig form
            phoneme_temp_str=getphoneme(g2p,temp_str)
            en_dic['<'+key+'-'+str(pair[1])+'>']=temp_str
            title_phoneme=title_phoneme.replace(temp_str,'<'+phoneme_temp_str+'>')
            title=title.replace(temp_str,'<'+key+'-'+str(pair[1])+'>')
    return [title,en_dic,title_phoneme,nltk]
  
def getkbtemplate(bt,g2p,title):
    title = re.sub(r'\u2019', "'", title)
    #Already has the cleaned/word_tokenized version
    title=title.replace('\n',' ')
    title_phoneme=title
    word_lst,ind_dic=bt.isinTrie(title)
    nlst=Word_Tokenize(Clean_Text(title))
    new_sen=title.strip()#cleans up sentence
    
    for key in ind_dic:
        ind_lst=ind_dic[key]
        for pair in ind_lst:
            temp_str=''
            for index in range(pair[0][0],pair[0][1]+1):
                if not nlst[index][0].isalnum():
                    temp_str=temp_str[:-1]
                temp_str+=nlst[index]+' '
            temp_str=temp_str[:-1]#getting rid of last space
            phoneme_temp_str=getphoneme(g2p,temp_str)
            title_phoneme=title.replace(temp_str,'<'+phoneme_temp_str+'>')
            title=title.replace(temp_str,'this '+key)

    insensitive_the_this=re.compile(re.escape('the this'),re.IGNORECASE)
    title=insensitive_the_this.sub('this',title)

    if title[0:4]=='this':
        title=title.replace('t','T',1)

    title=title.replace(' .','.')

    return (title,title_phoneme)

def run_dstc9_log(bt,data):
    #funciton works with: dstc9 logs, logs_eval, and dstc10 task1 + task2 logs.
    g2p=G2p()
    n=0
    ans_dic={}
    for dialogue in data:
        for section in dialogue:
            if section['text'] in ans_dic:
                info_lst=ans_dic[section['text']]
                section['template']=info_lst[0]
                section['id_map']=info_lst[1]
                section['phoneme_template']=info_lst[2]
                section['text_nltk']=info_lst[3]
                print('\t\t\tDUPLICATE')
            else:
                info_lst=gettemplate_wmap(bt,g2p,section['text'])
                section['template']=info_lst[0]
                section['id_map']=info_lst[1]
                section['phoneme_template']=info_lst[2]
                section['text_nltk']=info_lst[3]
                ans_dic[section['text']]=info_lst
                print('\t\t\tNEW')
                print(info_lst[0],info_lst[1],info_lst[2],info_lst[3],sep='\n')
            n+=1
            print('line/execution #',n,'--------------')
    return data

def run_dstc9_kb(bt,dic):
    #function works with: dstc9 kb, kb_eval, and dstc10 task2 kb. dstc10 task1 db has no doc entries (pure info)
    g2p=G2p()
    n=0
    for cat in dic:
        for num in dic[cat]:
            entity=dic[cat][num]['name']
            if not entity:#skip the taxi stuff
                continue
            entity_phoneme=getphoneme(g2p,entity)
            dic[cat][num]['phoneme_name']=entity_phoneme
            sen_dic=dic[cat][num]['docs']
            for entry in sen_dic:

                found_in_title=False
                found_in_body=False

                title=Clean_Text(sen_dic[entry]['title']).replace('\n',' ')
                title = re.sub(r'\u2019', "'", title)
                if entity=="ROSA'S BED AND BREAKFAST" and entry=='30':
                    print(title)
                    time.sleep(10)
                body=Clean_Text(sen_dic[entry]['body']).replace('\n',' ')
                body = re.sub(r'\u2019', "'", body)

                title_nltk=" ".join(Word_Tokenize(title))
                body_nltk=" ".join(Word_Tokenize(body))
                sen_dic[entry]['title_nltk']=title_nltk
                sen_dic[entry]['body_nltk']=body_nltk

                title_l=title.lower()
                if entity=="ROSA'S BED AND BREAKFAST" and entry=='30':
                    print(title_l)
                    time.sleep(10)
                body_l=body.lower()
                insensitive_entity = re.compile(re.escape(entity), re.IGNORECASE)
                
                if entity.lower() in title_l:
                    rtitle=insensitive_entity.sub('this '+cat,title)
                    ptitle_phoneme=insensitive_entity.sub('<'+entity_phoneme+'>',title)
                    if entity=="ROSA'S BED AND BREAKFAST" and entry=='30':
                        print(title)
                        print(rtitle)
                        time.sleep(10)
                    found_in_title=True
                if entity.lower() in body_l:
                    rbody=insensitive_entity.sub('this '+cat,body)
                    pbody_phoneme=insensitive_entity.sub('<'+entity_phoneme+'>',body)
                    found_in_body=True
                #secondary process: strings that has edit distance 2 (if len 4~5) or 3 (if len>8) will also be matched/replaced
                if not found_in_title:
                    rtitle,ptitle_phoneme=getkbtemplate(bt,g2p,title)
                    if rtitle!=title:
                        found_in_title=True
                if not found_in_body:
                    rbody,pbody_phoneme=getkbtemplate(bt,g2p,body)
                    if rbody!=body:
                        found_in_body=True
                
                subs_lst=['this location','your location','the location',
                          'your site','this site','the site',
                          'your restaurant','this restaurant','the restaurant',
                          'the property','this property','your property',
                          'your hotel','this hotel','the hotel',
                          'your facility','this facility','the facility',
                          'your bar','this bar','the bar',
                          'there?','here?','there.','here.']

                if not found_in_title:#still not found? replace/add in there for reference
                    new_title=title
                    for subs in subs_lst:#try to find stuff from list above to replace
                        insensitive_repl = re.compile(re.escape(subs), re.IGNORECASE)
                        if found_in_title==True:
                            break
                        if insensitive_repl.search(title)!=None:
                            if subs=='there?' or subs=='here?':
                                new_title=insensitive_repl.sub('at '+entity+'?',title)
                                rtitle=insensitive_repl.sub('at this '+cat+'?',title)
                                ptitle_phoneme=insensitive_repl.sub('at <'+entity_phoneme+'>?',title)
                            else:
                                new_title=insensitive_repl.sub(entity,title)
                                rtitle=insensitive_repl.sub('this '+cat,title)
                                ptitle_phoneme=insensitive_repl.sub('<'+entity_phoneme+'>',title)
                            found_in_title=True
                            new_title=new_title.replace('The this','This',1)
                            new_title=new_title.replace('the this','This',1)
                            if new_title[0:4]=='this':
                                new_title=new_title.replace('this','This',1)
                            sen_dic[entry]['title_mod']=new_title
                    if not found_in_title:#ADD as last resort
                        last=title[-1]
                        if not last.isalnum():#if last is a punc
                            new_title=title.replace(last,' at '+entity+last,1)
                            rtitle=title.replace(last,' at this '+cat+last,1)
                            ptitle_phoneme=title.replace(last,' at <'+entity_phoneme+'>'+last,1)
                        else:#if not default as a question
                            new_title=title+' at '+entity+'?'
                            rtitle=title+' at this '+cat+'?'
                            ptitle_phoneme=title+' at <'+entity_phoneme+'>?'
                        new_title=new_title.replace('The this','This',1)
                        new_title=new_title.replace('the this','This',1)
                        if new_title[0:4]=='this':
                            new_title=new_title.replace('this','This',1)
                        sen_dic[entry]['title_mod']=new_title
                    #print('title: '+rtitle,'title_mod: '+new_title,'title_phon: '+ptitle_phoneme,sep='\n')
                rtitle=rtitle.replace('The this','This',1)
                rtitle=rtitle.replace('the this','This',1)
                if rtitle[0:4]=='this':
                    rtitle=rtitle.replace('this','This',1)
                ptitle_phoneme=ptitle_phoneme.replace('The this','This',1)
                ptitle_phoneme=ptitle_phoneme.replace('the this','This',1)
                if ptitle_phoneme[0:4]=='this':
                    ptitle_phoneme=ptitle_phoneme.replace('this','This',1)
                
                sen_dic[entry]['title_template']=rtitle
                sen_dic[entry]['title_phoneme']=ptitle_phoneme#comment out right now
                if not found_in_body:
                    new_body=body
                    for subs in subs_lst:
                        insensitive_repl = re.compile(re.escape(subs), re.IGNORECASE)
                        if found_in_body==True:
                            break
                        if insensitive_repl.search(body)!=None:
                            if subs=='there.' or subs=='here.':
                                new_body=insensitive_repl.sub('at '+entity+'.',body)
                                rbody=insensitive_repl.sub('at this '+cat+'.',body)
                                pbody_phoneme=insensitive_repl.sub('at <'+entity_phoneme+'>.',body)
                            else:
                                new_body=insensitive_repl.sub(entity,body)
                                rbody=insensitive_repl.sub('this '+cat,body)
                                pbody_phoneme=insensitive_repl.sub('<'+entity_phoneme+'>',body)
                            found_in_body=True
                            new_body=new_body.replace('The this','This',1)
                            new_body=new_body.replace('the this','This',1)
                            if new_body[0:4]=='this':
                                new_body=new_body.replace('this','This',1)
                            sen_dic[entry]['body_mod']=new_body
                    if not found_in_body:#ADD as last resort
                        last=body[-1]
                        if not last.isalnum():#if last is a punc
                            new_body=body.replace(last,' at '+entity+last,1)
                            rbody=body.replace(last,' at this '+cat+last,1)
                            pbody_phoneme=body.replace(last,' at <'+entity_phoneme+'>'+last,1)
                        else:#if not default as full stop
                            new_body=body+' at '+entity+'.'
                            rbody=body+' at this '+cat+'.'
                            pbody_phoneme=body+' at <'+entity_phoneme+'>.'
                        new_body=new_body.replace('The this','This',1)
                        new_body=new_body.replace('the this','This',1)
                        if new_body[0:4]=='this':
                            new_body=new_body.replace('this','This',1)
                        sen_dic[entry]['body_mod']=new_body
                        
                rbody=rbody.replace('The this','This',1)
                rbody=rbody.replace('the this','This',1)
                if rbody[0:4]=='this':
                    rbody=rbody.replace('this','This',1)
                pbody_phoneme=pbody_phoneme.replace('The this','This',1)
                pbody_phoneme=pbody_phoneme.replace('the this','This',1)
                if pbody_phoneme[0:4]=='this':
                    pbody_phoneme=pbody_phoneme.replace('this','This',1)
                sen_dic[entry]['body_template']=rbody
                sen_dic[entry]['body_phoneme']=pbody_phoneme
                n+=1
                print('line/execution #',n,'--------------')
    return dic

def run_labels(bt,lb):
    g2p=G2p()
    n=0
    ans_dic={}
    for entry in lb:
        if entry['target']==True:
            text=entry['response']
            if text in ans_dic:
                info_lst=ans_dic[text]
                entry['response_template']=info_lst[0]
                entry['response_id_map']=info_lst[1]
                entry['response_phoneme']=info_lst[2]
                entry['response_nltk']=info_lst[3]
                print('\t\t\tDUPLICATE')
                continue
            info_lst=gettemplate_wmap(bt,g2p,text)
            ans_dic[text]=info_lst
            entry['response_template']=info_lst[0]
            entry['response_id_map']=info_lst[1]
            entry['response_phoneme']=info_lst[2]
            entry['response_nltk']=info_lst[3]
            print('\t\t\tNEW')
            print(info_lst[0],info_lst[1],info_lst[2],info_lst[3],sep='\n')
            n+=1
            print('line/execution #',n,'--------------')
    return lb


def main():
    '''
    kb=open('knowledge_mod.json','r')
    dic=json.load(kb)

    bt=BabyTrie()
    bt.initialize(dic)
    kb.close()
    title='Alpha-Milton Guest House'
    res=gettemplate_wmap(bt,title)
    print(res[0],res[1],res[2],sep='\n')
    #print(res)
    '''
    parser=argparse.ArgumentParser(description='DSTC10 Data Preprocessing (Summer 2021) For Wilson. Error Correction with clean text')
    parser.add_argument("--testmode",type=bool,default=False,help='set True if testing')
    parser.add_argument("--quick_selection",type=float,default=0,help='*0 to skip*  1.1: dstc9_log; 1.2: dstc9_kb; 2.1: dstc9_log_eval; 2.2: dstc9_kb_eval; 3:dstc10_task1 logs; 4:dstc10_task2 logs')
    parser.add_argument("--log_file",type=str,default='logs.json',help="input log file to be templated")
    parser.add_argument("--knowledge_file",type=str,default='knowledge.json',help="input knowledge file")
    parser.add_argument("--template_sel",type=float,default=1.2,help="select 1.1: template logs (not dstc10 task1); 1.2: template kb; 2: template dstc10 task1 logs")
    parser.add_argument("--output_file",type=str,default='output.txt',help="output templated file")
    
    args = parser.parse_args()

    if args.quick_selection==7:
        g2p=G2p()
        key_dic=open('keyword.txt','r')
        ans_dic={}
        out=open('phoneme_keyword.json','w')
        n=0
        for word in key_dic.read().split():
            if word in ans_dic:
                print('\t\t\tDUPLICATE')
                continue
            phon_word=getphoneme(g2p,word)
            ans_dic[word]=phon_word
            print('\t\t\tNEW')
            print(word,' ---> ',phon_word)
            n+=1
            print('line/execution #',n,'--------------')
        print('Dumping......')
        json.dump(ans_dic,out,indent=4)
        print('Done.')

    if args.quick_selection==6:
        g2p=G2p()
        print('Collecting Phoneme dictionary......')
        #FILE NEEDS TO BE CHANGED IF RUNNING ON HPC: CHANGE QUICK SELECTION?
        logs=['logs_mod.json',
              '/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data/val/logs.json',
              '/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task2/data/val/logs.json',
              '/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task1/data/val/logs.json'
              ]
        kb='/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data_eval/knowledge.json'
        out=open('phoneme_dic.json','w')
        phoneme_dic={}
        for i in range(len(logs)):
            n=0
            print('Processing log #',i,'......')
            log=open(logs[i],'r')
            data=json.load(log)
            for dialogue in data:
                for section in dialogue:
                    ws_lst=section['text'].split()
                    for word in ws_lst:
                        if word in phoneme_dic:
                            print('\t\t\tDUPLICATE')
                            continue
                        phon_word=getphoneme(g2p,word)
                        phoneme_dic[word]=phon_word
                        print('\t\t\tNEW')
                        print(word,' ---> ',phon_word)
                    n+=1
                    print('line/execution #',n,'--------------')
            log.close()
        n=0
        print('Processing kb......')
        knowledge=open(kb,'r')
        dic=json.load(knowledge)
        for cat in dic:
            for num in dic[cat]:
                entity=dic[cat][num]['name']
                sen_dic=dic[cat][num]['docs']
                if not entity:#skip the taxi stuff
                    continue
                enti_lst=entity.split()
                for word in enti_lst:
                    if word in phoneme_dic:
                        print('\t\t\tDUPLICATE')
                        continue
                    phon_word=getphoneme(g2p,word)
                    phoneme_dic[word]=phon_word
                    print('\t\t\tNEW')
                    print(word,' ---> ',phon_word)
                for entry in sen_dic:
                    title=sen_dic[entry]['title']
                    body=sen_dic[entry]['body']
                    title_lst=title.split()
                    body_lst=body.split()
                    for word in title_lst:
                        if word in phoneme_dic:
                            print('\t\t\tDUPLICATE')
                            continue
                        phon_word=getphoneme(g2p,word)
                        phoneme_dic[word]=phon_word
                        print('\t\t\tNEW')
                        print(word,' ---> ',phon_word)
                    for word in body_lst:
                        if word in phoneme_dic:
                            print('\t\t\tDUPLICATE')
                            continue
                        phon_word=getphoneme(g2p,word)
                        phoneme_dic[word]=phon_word
                        print('\t\t\tNEW')
                        print(word,' ---> ',phon_word)
                    n+=1
                    print('line/execution #',n,'--------------')
        knowledge.close()
        print('Dumping......')
        json.dump(phoneme_dic,out,indent=4)
        out.close()
        print('Done.')

    if args.quick_selection==5:#~1hr 30 min
        print('Running labels batch......')
        labels=['/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data/train/labels.json',
                '/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data/val/labels.json',
                '/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data_eval/test/labels.json',
                '/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task2/data/val/labels.json'
                
        ]
        kbs=['knowledge_mod.json',
             '/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data_eval/knowledge.json'
        ]
        outs=['DSTC9_task1_train_labels_phon_templates.json',
              'DSTC9_task1_val_labels_phon_templates.json',
              'DSTC9_task1_eval_labels_phon_templates.json',
              'DSTC10_track2_task2_labels_phon_templates.json'
        ]
        for i in range(len(labels)):
            out=open(outs[i],'w')
            label=open(labels[i],'r')
            lb=json.load(label)

            bt=BabyTrie()#setting up bt based on knowledge base
            if i<=1:
                kb=open(kbs[0],'r')
                dic=json.load(kb)
                bt.initialize(dic)
                print('Trie successfully initialized.')
            else:
                kb=open(kbs[1],'r')
                dic=json.load(kb)
                bt.initialize_w_ref(dic)
                print('Ref_Trie successfully initialized.')
            label.close()
            kb.close()
            new_data=run_labels(bt,lb)

            print('dumping...')
            json.dump(new_data,out,indent=4)
            out.close()
            print('Done.')

    elif args.quick_selection==1.1:
        out=open('DSTC9_task1_train_log_phon_templates.json','w')
        logs=open('logs_mod.json','r')
        data=json.load(logs)
        kb=open('knowledge_mod.json','r')
        dic=json.load(kb)

        bt=BabyTrie()#setting up bt based on knowledge base
        bt.initialize(dic)
        print('Trie successfully initialized.')
        logs.close()
        kb.close()
        new_data=run_dstc9_log(bt,data)

        print('dumping...')
        json.dump(new_data,out,indent=4)
        out.close()
        print('Done.')
    elif args.quick_selection==1.3:# 101,649 lines
        out=open('DSTC9_task1_val_log_phon_templates.json','w')
        logs=open('/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data/val/logs.json','r')
        data=json.load(logs)
        kb=open('knowledge_mod.json','r')
        dic=json.load(kb)

        bt=BabyTrie()#setting up bt based on knowledge base
        bt.initialize(dic)
        print('Trie successfully initialized.')
        logs.close()
        kb.close()
        new_data=run_dstc9_log(bt,data)

        print('dumping...')
        json.dump(new_data,out,indent=4)
        out.close()
        print('Done.')
    elif args.quick_selection==1.2:
        out=open('DSTC9_task1_kb_phon_templates.json','w')
        kb=open('knowledge_mod.json','r')
        dic=json.load(kb)

        bt=BabyTrie()#setting up bt based on knowledge base
        bt.initialize(dic)
        print('Trie successfully initialized.')
        kb.close()
        new_data=run_dstc9_kb(bt,dic)

        print('dumping...')
        json.dump(new_data,out,indent=4)
        out.close()
        print('Done.')
    elif args.quick_selection==2.1:
        out=open('DSTC10_task1_log_eval_phon_templates.json','w')
        logs=open('/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task1/data/test/logs.json','r')
        data=json.load(logs)
        kb=open('/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task2/data/knowledge.json','r')
        dic=json.load(kb)

        bt=BabyTrie()#setting up bt based on knowledge base
        bt.initialize_w_ref(dic)
        print('Trie successfully initialized.')
        logs.close()
        kb.close()
        new_data=run_dstc9_log(bt,data)

        print('dumping...')
        json.dump(new_data,out,indent=4)
        out.close()
        print('Done.')
    elif args.quick_selection==2.2:
        out=open('DSTC10_task2_kb_eval_phon_templates.json','w')
        kb=open('/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data_eval/knowledge.json','r')
        dic=json.load(kb)

        bt=BabyTrie()#setting up bt based on knowledge base
        bt.initialize_w_ref(dic)
        print('Trie successfully initialized.')
        kb.close()
        new_data=run_dstc9_kb(bt,dic)

        print('dumping...')
        json.dump(new_data,out,indent=4)
        out.close()
        print('Done.')
    elif args.quick_selection==3:
        out=open('DSTC10_task1_log_phon_templates.json','w')
        logs=open('/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task1/data/val/logs.json','r')
        data=json.load(logs)
        kb=open('/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task1/data/db.json','r')
        dic=json.load(kb)

        bt=BabyTrie()#setting up bt based on knowledge base
        bt.sp_initialize(dic)
        print('Special Trie successfully initialized.')
        logs.close()
        kb.close()
        new_data=run_dstc9_log(bt,data)

        print('dumping...')
        json.dump(new_data,out,indent=4)
        out.close()
        print('Done.')
    elif args.quick_selection==4:
        out=open('DSTC10_task2_log_eval_phon_templates.json','w')
        logs=open('/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task2/data/test/logs.json','r')
        data=json.load(logs)
        kb=open('/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task2/data/knowledge.json','r')
        dic=json.load(kb)

        bt=BabyTrie()#setting up bt based on knowledge base
        bt.initialize_w_ref(dic)
        print('Trie successfully initialized.')
        logs.close()
        kb.close()
        new_data=run_dstc9_log(bt,data)

        print('dumping...')
        json.dump(new_data,out,indent=4)
        out.close()
        print('Done.')

if __name__=='__main__':
    main()

Overwriting Process_clean_for_eval.py


In [ ]:
!cp ../extra_methods.py extra_methods.py

In [ ]:
!python Process_clean_for_eval.py --quick_selection 2.1

Streaming output truncated to the last 5000 lines.
line/execution # 84302 --------------
			DUPLICATE
line/execution # 84303 --------------
			DUPLICATE
line/execution # 84304 --------------
			DUPLICATE
line/execution # 84305 --------------
			DUPLICATE
line/execution # 84306 --------------
			DUPLICATE
line/execution # 84307 --------------
			DUPLICATE
line/execution # 84308 --------------
			DUPLICATE
line/execution # 84309 --------------
			DUPLICATE
line/execution # 84310 --------------
			DUPLICATE
line/execution # 84311 --------------
			DUPLICATE
line/execution # 84312 --------------
			DUPLICATE
line/execution # 84313 --------------
			DUPLICATE
line/execution # 84314 --------------
			DUPLICATE
line/execution # 84315 --------------
			DUPLICATE
line/execution # 84316 --------------
			DUPLICATE
line/execution # 84317 --------------
			DUPLICATE
line/execution # 84318 --------------
			DUPLICATE
line/execution # 84319 --------------
			DUPLICATE
line/execution # 84320 --------

# Preprocessing for Nathan

In [ ]:
!python preprocess_for_nathan.py --quick_selection 6
#3 not needed

In [ ]:
#@title DSTC10
%%writefile DSTC10_ner_test.py
#NOTE: lines in logs.json/logs_mod.json = 768,500
#NOTE: lines in logs_eval.json/logs_mod.json = 36,091
import json
import re
import argparse
from BabyTrie_DSTC10_v2_test import BabyTrie
from extra_methods import convert_text2num#may have bugs such as for 'second one' = 3
from g2p_en import G2p

def getphoneme(text):
    g2p=G2p()
    ptext=g2p(text)
    string='-'.join(ptext)
    res=string.replace('- -',' ')
    return res

def gettemplate_wmap(bt,title):
    #title=convert_text2num(title).strip()
    word_lst,ind_dic=bt.isinTrie(title)
    nlst=[]
    new_sen=title.strip()#cleans up sentence
    plst=re.findall(r"[\w']+|[.,!?;]", new_sen)#premature list used for clean-up
    p=re.compile(r"^(\w+)(\'\w+)$")
    for i in range (len(plst)):
        if p.match(plst[i]):
            new_word1,new_word2=p.match(plst[i]).group(1),p.match(plst[i]).group(2)
            nlst.append(new_word1)
            nlst.append(new_word2)
        else:
            nlst.append(plst[i])
    en_dic={}
    for key in ind_dic:
        ind_lst=ind_dic[key]
        for pair in ind_lst:
            temp_str=''
            for index in range(pair[0][0],pair[0][1]+1):
                if not nlst[index][0].isalnum():
                    temp_str=temp_str[:-1]
                temp_str+=nlst[index]+' '
            temp_str=temp_str[:-1]#getting rid of last space
            en_dic['<'+key+'-'+str(pair[1])+'>']=temp_str
            title=title.replace(temp_str,'<'+key+'-'+str(pair[1])+'>')
    return (title,en_dic)
  
def getkbtemplate(bt,title):
    word_lst,ind_dic=bt.isinTrie(title)
    nlst=[]
    new_sen=title.strip()#cleans up sentence
    plst=re.findall(r"[\w']+|[.,!?;]", new_sen)#premature list used for clean-up
    p=re.compile(r"^(\w+)(\'\w+)$")
    for i in range (len(plst)):
        if p.match(plst[i]):
            new_word1,new_word2=p.match(plst[i]).group(1),p.match(plst[i]).group(2)
            nlst.append(new_word1)
            nlst.append(new_word2)
        else:
            nlst.append(plst[i])
    for key in ind_dic:
        ind_lst=ind_dic[key]
        for pair in ind_lst:
            temp_str=''
            for index in range(pair[0][0],pair[0][1]+1):
                if not nlst[index][0].isalnum():
                    temp_str=temp_str[:-1]
                temp_str+=nlst[index]+' '
            temp_str=temp_str[:-1]#getting rid of last space
            title=title.replace(temp_str,'this '+key)

    insensitive_the_this=re.compile(re.escape('the this'),re.IGNORECASE)
    title=insensitive_the_this.sub('this',title)

    if title[0:4]=='this':
        title=title.replace('t','T',1)

    title=title.replace(' .','.')

    return (title)

def run_dstc9_log(bt,data):
    #funciton works with: dstc9 logs, logs_eval, and dstc10 task1 + task2 logs.
    n=0
    for dialogue in data:
        for section in dialogue:
            phoneme=getphoneme(section['text'])
            template,map=gettemplate_wmap(bt,section['text'])
            section['template']=template
            section['id_map']=map
            section['phoneme']=phoneme
            n+=1
            print('line/execution #',n)
    return data

def run_dstc9_kb(bt,dic):
    #function works with: dstc9 kb, kb_eval, and dstc10 task2 kb. dstc10 task1 db has no doc entries (pure info)
    n=0
    for cat in dic:
        for num in dic[cat]:
            entity=dic[cat][num]['name']
            if not entity:#skip the taxi stuff
                continue
            sen_dic=dic[cat][num]['docs']
            for entry in sen_dic:

                found_in_title=False
                found_in_body=False

                title=sen_dic[entry]['title']
                title_phoneme=getphoneme(title)
                body=sen_dic[entry]['body']
                body_phoneme=getphoneme(body)
                title_l=title.lower()
                body_l=body.lower()
                insensitive_entity = re.compile(re.escape(entity), re.IGNORECASE)
                
                if entity.lower() in title_l:
                    ptitle=insensitive_entity.sub('this '+cat,title)
                    found_in_title=True
                if entity.lower() in body_l:
                    pbody=insensitive_entity.sub('this '+cat,body)
                    found_in_body=True
                #secondary process: strings that has edit distance 2 (if len 4~5) or 3 (if len>8) will also be matched/replaced
                if not found_in_title:
                    ptitle=getkbtemplate(bt,title)
                if not found_in_body:
                    pbody=getkbtemplate(bt,body)
                
                rtitle=ptitle
                rbody=pbody
                sen_dic[entry]['title_template']=rtitle
                sen_dic[entry]['title_phoneme']=title_phoneme
                sen_dic[entry]['body_template']=rbody
                sen_dic[entry]['body_phoneme']=body_phoneme
                print('line: ',n)
                n+=1
    return dic

def run_labels(bt,lb):
    g2p=G2p()
    n=0
    for entry in lb:
        if entry['target']==True:
            text=entry['response']
            phoneme=getphoneme(text)
            template,map=gettemplate_wmap(bt,text)
            entry['response_template']=template
            entry['response_id_map']=map
            entry['response_phoneme']=phoneme
            n+=1
            print('line/execution #',n)
    return lb


def main():
    '''
    kb=open('knowledge_mod.json','r')
    dic=json.load(kb)

    bt=BabyTrie()
    bt.initialize(dic)
    kb.close()
    title='Does the Worth House offer a free breakfast?'
    res=getkbtemplate(bt,title)
    #print(res[0],res[1],res[2],sep='\n')
    print(res)
    '''
    parser=argparse.ArgumentParser(description='DSTC10 Data Preprocessing (Summer 2021)')
    parser.add_argument("--testmode",type=bool,default=False,help='set True if testing')
    parser.add_argument("--quick_selection",type=float,default=0,help='*0 to skip*  1.1: dstc9_log; 1.2: dstc9_kb; 2.1: dstc9_log_eval; 2.2: dstc9_kb_eval; 3:dstc10_task1 logs; 4:dstc10_task2 logs')
    parser.add_argument("--log_file",type=str,default='logs.json',help="input log file to be templated")
    parser.add_argument("--knowledge_file",type=str,default='knowledge.json',help="input knowledge file")
    parser.add_argument("--template_sel",type=float,default=1.2,help="select 1.1: template logs (not dstc10 task1); 1.2: template kb; 2: template dstc10 task1 logs")
    parser.add_argument("--output_file",type=str,default='output.txt',help="output templated file")
    
    args = parser.parse_args()

    if args.quick_selection==5:
        print('Running labels batch......')
        labels=['/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data/train/labels.json',
                '/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data/val/labels.json',
                '/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data_eval/test/labels.json',
                '/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task2/data/val/labels.json'
                
        ]
        kbs=['knowledge_mod.json',
             '/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data_eval/knowledge.json'
        ]
        outs=['DSTC9_task1_train_labels_template_PHONEME.json',
              'DSTC9_task1_val_labels_template_PHONEME.json',
              'DSTC9_task1_eval_labels_template_PHONEME.json',
              'DSTC9_track2_task2_labels_template_PHONEME.json'
        ]
        for i in range(len(labels)):
            out=open(outs[i],'w')
            label=open(labels[i],'r')
            lb=json.load(label)

            bt=BabyTrie()#setting up bt based on knowledge base
            if i<=1:
                kb=open(kbs[0],'r')
                dic=json.load(kb)
                bt.initialize(dic)
                print('Trie successfully initialized.')
            else:
                kb=open(kbs[1],'r')
                dic=json.load(kb)
                bt.initialize_w_ref(dic)
                print('Ref_Trie successfully initialized.')
            label.close()
            kb.close()
            new_data=run_labels(bt,lb)

            print('dumping...')
            json.dump(new_data,out,indent=4)
            out.close()
            print('Done.')

    elif args.quick_selection==1.1:
        out=open('DSTC9_task1_train_log_templates.json','w')
        logs=open('logs_mod.json','r')
        data=json.load(logs)
        kb=open('knowledge_mod.json','r')
        dic=json.load(kb)

        bt=BabyTrie()#setting up bt based on knowledge base
        bt.initialize(dic)
        print('Trie successfully initialized.')
        logs.close()
        kb.close()
        new_data=run_dstc9_log(bt,data)

        print('dumping...')
        json.dump(new_data,out,indent=4)
        out.close()
        print('Done.')
    elif args.quick_selection==1.3:
        out=open('DSTC9_task1_val_log_templates.json','w')
        logs=open('/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data/val/logs.json','r')
        data=json.load(logs)
        kb=open('knowledge_mod.json','r')
        dic=json.load(kb)

        bt=BabyTrie()#setting up bt based on knowledge base
        bt.initialize(dic)
        print('Trie successfully initialized.')
        logs.close()
        kb.close()
        new_data=run_dstc9_log(bt,data)

        print('dumping...')
        json.dump(new_data,out,indent=4)
        out.close()
        print('Done.')
    elif args.quick_selection==1.2:
        out=open('DSTC9_task1_kb_templates.json','w')
        kb=open('knowledge_mod.json','r')
        dic=json.load(kb)

        bt=BabyTrie()#setting up bt based on knowledge base
        bt.initialize(dic)
        print('Trie successfully initialized.')
        kb.close()
        new_data=run_dstc9_kb(bt,dic)

        print('dumping...')
        json.dump(new_data,out,indent=4)
        out.close()
        print('Done.')
    elif args.quick_selection==2.1:
        out=open('DSTC9_task1_log_eval_templates.json','w')
        logs=open('/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data_eval/test/logs.json','r')
        data=json.load(logs)
        kb=open('/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data_eval/knowledge.json','r')
        dic=json.load(kb)

        bt=BabyTrie()#setting up bt based on knowledge base
        bt.initialize_w_ref(dic)
        print('Trie successfully initialized.')
        logs.close()
        kb.close()
        new_data=run_dstc9_log(bt,data)

        print('dumping...')
        json.dump(new_data,out,indent=4)
        out.close()
        print('Done.')
    elif args.quick_selection==2.2:
        out=open('DSTC9_task1_kb_eval_templates.json','w')
        kb=open('/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data_eval/knowledge.json','r')
        dic=json.load(kb)

        bt=BabyTrie()#setting up bt based on knowledge base
        bt.initialize(dic)
        print('Trie successfully initialized.')
        kb.close()
        new_data=run_dstc9_kb(bt,dic)

        print('dumping...')
        json.dump(new_data,out,indent=4)
        out.close()
        print('Done.')
    elif args.quick_selection==3:
        out=open('DSTC10_task1_log_templates.json','w')
        logs=open('/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task1/data/val/logs.json','r')
        data=json.load(logs)
        kb=open('/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task1/data/db.json','r')
        dic=json.load(kb)

        bt=BabyTrie()#setting up bt based on knowledge base
        bt.sp_initialize(dic)
        print('Special Trie successfully initialized.')
        logs.close()
        kb.close()
        new_data=run_dstc9_log(bt,data)

        print('dumping...')
        json.dump(new_data,out,indent=4)
        out.close()
        print('Done.')
    elif args.quick_selection==4:
        out=open('DSTC10_task2_log_templates.json','w')
        logs=open('/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task2/data/val/logs.json','r')
        data=json.load(logs)
        kb=open('/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task2/data/knowledge.json','r')
        dic=json.load(kb)

        bt=BabyTrie()#setting up bt based on knowledge base
        bt.initialize_w_ref(dic)
        print('Trie successfully initialized.')
        logs.close()
        kb.close()
        new_data=run_dstc9_log(bt,data)

        print('dumping...')
        json.dump(new_data,out,indent=4)
        out.close()
        print('Done.')
    
    elif not args.testmode:
        out=open(args.output_file,'w')
        logs=open(args.log_file,'r')
        data=json.load(logs)
        kb=open(args.knowledge_file,'r')
        dic=json.load(kb)

        bt=BabyTrie()#setting up bt based on knowledge base
        if args.template_sel==1.1:
            bt.initialize(dic)
            print('Trie successfully initialized.')
            new_data=run_dstc9_log(bt,data)
        elif args.template_sel==1.2:
            bt.initialize(dic)
            print('Trie successfully initialized.')
            new_data=run_dstc9_kb(bt,dic)
        elif args.template_sel==2:
            bt.sp_initialize(dic)
            print('Special Trie successfully initialized.')
            new_data=run_dstc9_log(bt,data)
        
        logs.close()
        kb.close()

        print('dumping...')
        json.dump(new_data,out,indent=4)
        out.close()
        print('Done.')
    
    elif args.testmode:
        print('Using dstc10 task1 db to initialize bt:')
        kb=open('/content/drive/MyDrive/Research/ner/alexa-with-dstc10-track2-dataset/task1/data/db.json','r')
        dic=json.load(kb)
        bt=BabyTrie()#setting up bt based on knowledge base
        bt.sp_initialize(dic)
        print('(sp) Trie successfully initialized.')
        mode=input('Input testing mode; 1 for log style and 2 for kb style')
        sen=input('Input testing sentence. Input exit to stop.')
        while sen!='exit':
            if mode==1:
                print('Running test as |dstc9/dstc10| logs.json example...')
                print('\n')
                template,map=gettemplate_wmap(bt,sen)
                print('Template: ',template)
                print('Map: ',map)
            elif mode==2:
                print('Running test as |dstc9/dstc10_task2| knowledge.json example...')
                print('\n')
                template=getkbtemplate(bt,sen)
                print('Template: ',template)
            mode=input('Input another testing mode; 1 for log style and 2 for kb style')
            sen=input('Input another testing sentence. Input exit to stop.')
    
if __name__=='__main__':
    main()

Overwriting DSTC10_ner_test.py


In [ ]:
!python DSTC10_ner_test.py --quick_selection 5

In [ ]:
import num2words
num2words('')

     |████████████████████████████████| 101 kB 2.3 MB/s 


In [ ]:
#@title Extra Methods for DSTC10
#%%writefile extra_methods.py
import re

def simple_num2words(int):
    lst=['zero','one','two','three','four','five','six','seven','eight',
         'nine']
    return lst[int]

def is_number(x):
    if type(x) == str:
        x = x.replace(',', '')
    try:
        float(x)
    except:
        return False
    return True

def convert_text2num (textnum, numwords={}):
    units = [
        'zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight',
        'nine', 'ten', 'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen',
        'sixteen', 'seventeen', 'eighteen', 'nineteen',
    ]
    tens = ['', '', 'twenty', 'thirty', 'forty', 'fifty', 'sixty', 'seventy', 'eighty', 'ninety']
    scales = ['hundred', 'thousand', 'million', 'billion', 'trillion']
    ordinal_words = {'first':1, 'second':2, 'third':3, 'fifth':5, 'sixth':6, 'seventh':7, 'eighth':8, 'ninth':9, 'twelfth':12}
    
    #ordinal_endings = [('ieth', 'y'), ('th', '')]
    
    if not numwords:
        numwords['and'] = (1, 0)
        for idx, word in enumerate(units): numwords[word] = (1, idx)
        for idx, word in enumerate(tens): numwords[word] = (1, idx * 10)
        for idx, word in enumerate(scales): numwords[word] = (10 ** (idx * 3 or 2), 0)

    #textnum = textnum.replace('-', ' ')

    current = result = 0
    curstring = ''
    onnumber = False
    lastunit = False
    lastscale = False

    def is_numword(x):
        if is_number(x):
            return True
        if word in numwords:
            return True
        return False

    def from_numword(x):
        if is_number(x):
            scale = 0
            increment = int(x.replace(',', ''))
            return scale, increment
        return numwords[x]

    skip_next = False
    word_lst=re.findall(r"[\w']+|[.,!?;]", textnum)
    last=len(word_lst)-1
    for i in range(len(word_lst)):
        word=word_lst[i]
        if skip_next:
            curstring += word + ' '
            skip_next = False
            continue
        if word in ordinal_words:
            if word_lst[i+1] in units:
                curstring += word + ' '
                skip_next = True
                continue
            scale, increment = (1, ordinal_words[word])
            current = current * scale + increment
            if scale > 100:
                result += current
                current = 0
            onnumber = True
            lastunit = False
            lastscale = False
        else:
            '''
            for ending, replacement in ordinal_endings:
                if word.endswith(ending):
                    word = "%s%s" % (word[:-len(ending)], replacement)
            '''
            if (not is_numword(word)) or (word == 'and' and not lastscale):
                if onnumber:
                    # Flush the current number we are building
                        curstring += repr(result + current)+' '
                if (not word.isalnum()) and (len(word)==1):
                    curstring=curstring[:-1]
                curstring += word+" "
                result = current = 0
                onnumber = False
                lastunit = False
                lastscale = False
            else:
                scale, increment = from_numword(word)
                onnumber = True

                if lastunit and (word not in scales):                                                                                                                                                                                                                                         
                    # Assume this is part of a string of individual numbers to                                                                                                                                                                                                                
                    # be flushed, such as a zipcode "one two three four five"                                                                                                                                                                                                                 
                    curstring += repr(result + current)                                                                                                                                                                                                                                    
                    result = current = 0                                                                                                                                                                                                                                                      

                if scale > 1:                                                                                                                                                                                                                                                                 
                    current = max(1, current)                                                                                                                                                                                                                                                 

                current = current * scale + increment                                                                                                                                                                                                                                         
                if scale > 100:                                                                                                                                                                                                                                                               
                    result += current                                                                                                                                                                                                                                                         
                    current = 0                                                                                                                                                                                                                                                               

                lastscale = False                                                                                                                                                                                                              
                lastunit = False                                                                                                                                                
                if word in scales:                                                                                                                                                                                                             
                    lastscale = True                                                                                                                                                                                                         
                elif word in units:                                                                                                                                                                                                             
                    lastunit = True

    if onnumber:
        print(curstring, result, current, sep='\t')
        curstring += repr(result + current)

    return curstring


In [ ]:
!pip install compound-word-splitter
!pip install pyenchant
!apt-get install enchant

/bin/bash: apt-get install enchant: command not found


In [ ]:
import enchant
import splitter

splitter.split('seafood')

In [ ]:
#@title BABYTRIE
#%%writefile BabyTrie_DSTC10_v5.py
import re
import json
from fastDamerauLevenshtein import damerauLevenshtein as dl
from num2words import num2words
from simple_tokenize import Clean_Text
from simple_tokenize import Word_Tokenize
from extra_methods import simple_num2words

class BabyTrie:#baby version of Trie
    class TrieNode:#Node within a Trie
        def __init__(self,entity=None):
            self.children={}#dictionary of TrieNodes
            self.markers=[]#list of markers/categories
            self.end=False#the Node is leaf, or end of word
            self.entity=None#in case needed
            self.id=None
    
    def __init__(self):
        self.root=self.new_node()

    def new_node(self,word=None):
        #Creates TrieNode object with a given word
        return self.TrieNode(word)

    def restricted_insert(self,lst,cat,number):
        if len(lst) == 0:
            print("ERROR: empty NE in knowledge file")
            return
        if len(lst) == 1 and lst[0]==cat:
            print("only one elem in lst")
            return
        
        if lst[0]=='the':#the is ambiguous
            lst.pop(0)

        ptr=self.root
        for i in range(0,len(lst)):#for every elem but not the last (since it is the category)
            if lst[i] not in ptr.children.keys():#if already a key, skip; else add new
                new=self.new_node(lst[i])
                ptr.children[lst[i]]=new
            ptr=ptr.children[lst[i]]
        #if (len(lst)==2 and lst[-2] in ['good','ask','page']):#we dont want to treat good or ask as an entity (see test.txt)
            #ptr.end=False
        if (len(lst)==3 and lst[-3]=='cable' and lst[-2]=='car'):
            ptr.end=False
        else:
            ptr.end=True#is end of word
            ptr.id=number
        if cat not in ptr.markers:#Only adds new categories to the list of markers 
            ptr.markers.append(cat)
    
    def insert(self,lst,cat,number):
        '''
        Given a list of words that make up a Named Entity and its category,
        inserts the Entity into the Trie.

        Note: the structure resembles a tree, i.e. words as nodes, and at the end
        of the inserted word, the node is made a leaf node (end=True), and a Marker
        is added to the node's list of markers.
        -------------------------------------------------------------------
        Example:
        bt=BabyTrie()
        possible_NE=['Jade','Garden']#let's say that this is a restaurant and id#1
        bt.insert(possible_NE,'restaurant',1)
        '''
        if len(lst) == 0:
            print("ERROR: empty NE in knowledge file")
            return
        if len(lst) == 1 and lst[0]==cat:
            print("only one elem in lst")
            return
        
        if lst[0]=='the':#the is ambiguous
            lst.pop(0)
        
        
        
        if lst[-1]!=cat: #['a','and','b'] + ['restaurant']
            lst.append(cat)
        '''
        for the sake of more possible matches:
        If inserted "Jade Garden" and it's a restaurant,
        "Jade Garden" will be recognized as an entity;
        and "Jade Garden Restaurant" will also be one.
        '''
        ptr=self.root
        for i in range(0,len(lst)-1):#for every elem but not the last (since it is the category)
            if lst[i] not in ptr.children.keys():#if already a key, skip; else add new
                new=self.new_node(lst[i])
                ptr.children[lst[i]]=new
            ptr=ptr.children[lst[i]]
        if (len(lst)==2 and lst[-2] in ['good','ask','page']):#we dont want to treat good or ask as an entity (see test.txt)
            ptr.end=False
        elif (len(lst)==3 and lst[-3]=='cable' and lst[-2]=='car'):
            ptr.end=False
        else:
            ptr.end=True#is end of word
            ptr.id=number
        if cat not in ptr.markers:#Only adds new categories to the list of markers 
            ptr.markers.append(cat)
        if lst[-1] not in ptr.children.keys():#avoid possible conflict with same name
            new=self.new_node(lst[-1])
            ptr.children[lst[-1]]=new#adds the category as one of the leaf node
        ptr=ptr.children[lst[-1]]
        ptr.end=True
        ptr.id=number
        if cat not in ptr.markers:#only adds category if not in list of markers
            ptr.markers.append(cat)
    
    def isinTrie(self,sen):
        stutter_match=False
        dl_mistakes=['train','want','american','lane','fees','fee','marina','wanna','canna','dinna','finna']
        #outfile for tracking log process
        dic={}#the uncleaned version of returned dictionary
        flag=False#=True when a phrase in the sentence does not match the trie anymore
        lst=[]#the list that will ultimately be returned

        new_sen=sen.lower().strip()#cleans up sentence

        lst=Word_Tokenize(Clean_Text(new_sen))
        
        last=len(lst)-1#keep track of last to ensure index does not go over limit
        #the returned list is completely formed at this point, we use it to generate dictionary
        rstart=0#starting position of NE
        rend=0#ending position of NE

        proceed=False
        for i in range(len(lst)):
            ptr=self.root
            if lst[i] in ptr.children:
                proceed=True
                rstart=i
                ptr=ptr.children[lst[i]]
            elif lst[i] not in ptr.children and lst[i] not in dl_mistakes:
                for word in ptr.children:
                    if len(lst[i])<=4:
                        dl_score=dl(lst[i],word,similarity=False,deleteWeight=2,insertWeight=2,replaceWeight=2)
                        bm=0
                    else:
                        dl_score=dl(lst[i],word,similarity=False,deleteWeight=1,insertWeight=2,replaceWeight=2)
                        bm=1
                    if dl_score<=bm:
                        proceed=True
                        rstart=i
                        ptr=ptr.children[word]
                        break
            if proceed:
                stutter_utt=0
                for j in range(i+1,len(lst)):
                    if lst[j] in ptr.children:
                        ptr=ptr.children[lst[j]]
                        rend=j
                        stutter_match=False
                    elif lst[j] not in ptr.children:
                        proceed=False
                        for word in ptr.children:
                            dl_score=dl(lst[j],word,similarity=False,deleteWeight=2,insertWeight=2,replaceWeight=2)
                            bm=1
                            if dl_score<=bm and lst[j].isalnum():
                                ptr=ptr.children[word]
                                rend=j
                                proceed=True
                                stutter_match=False
                                break
                        if (len(lst[j])<=3) and (lst[j].isalnum()) and (stutter_utt<1):#allow one mismatch (except for punc), could be stuttering
                            stutter_utt+=1
                            proceed=True
                            rend=j-1
                            stutter_match=True
                            #here rend not counted since if next is not matched, it will not be effected if the prev word is actually end of word
                        elif stutter_match:
                            rend=j-2
                            break
                        #print('for '+lst[i]+' stopped at '+lst[j])#degbug use
                        if not proceed:
                            rend=j-1
                            break

            if ptr.end:
                for i in range(len(ptr.markers)):
                    if (ptr.markers[i] in dic):#if already a key, add to value
                        if rstart>rend:
                            rend=rstart
                        dic[ptr.markers[i]].append(((rstart,rend),ptr.id))
                    else:#if not, make new entry
                        if rstart>rend:
                            rend=rstart
                        dic[ptr.markers[i]]=[((rstart,rend),ptr.id)]
                rstart=0
                rend=0
            else:
                continue
        print(dic)
        #cleaning up dictionary
        rdic={}#clean version of the dictionary that is ultimately returned
        for cat in dic.keys():
            rdic[cat]=[]
            rlst=sorted(dic[cat],key=lambda item:item[0][1]-item[0][0],reverse=True)
            #rdic[cat].append(rlst[0])
            for i in range(0,len(rlst)):
                rflag=True
                for rcat in rdic.keys():#make sure no overlap: ex. 'some hotel diner' > 'some hotel'
                    for elem in rdic[rcat]:
                        if (rlst[i][0][0]<elem[0][0] and rlst[i][0][1]<elem[0][0]):#ex. if we have (3,5) we can take (1,2)
                            continue
                        elif (rlst[i][0][0]>elem[0][1] and rlst[i][0][1]>elem[0][1]):#ex. if we have (3,5) we can take (6,7)
                            continue
                        else:# any equals, [0] or [1] will not work: ex. we have (3,5). cannot have (3,4) or (4,5).
                            if rlst[i][0][1]-rlst[i][0][0]==elem[0][1]-elem[0][0] and rlst[i][0][1]>elem[0][1]:
                                rdic[rcat].remove(elem)
                                continue
                            else:
                                rflag=False
                if rflag:
                    rdic[cat].append(rlst[i])
        del_lst=[]#cheap(costly) way to deal with empty dic entry
        for cat in rdic:
            if len(rdic[cat])==0:
                del_lst.append(cat)
        for elem in del_lst:
            del rdic[elem]
        return (lst,rdic)
        
    def initialize(self,dic):
        dig=False
        chain_entities={}#keeping track of chain restaurants' ids i.e. {'entity name':('cat','id1|id2|id3|...')}
        for cat in dic.keys():
            for elem in dic[cat]:
                record = dic[cat][elem]

                if ('name' in record and record['name'] is not None):
                    new_name=record['name'].lower().strip()#cleans up name
                    if new_name == cat:
                        print('exceptional entity---------------------')
                        continue
                    lst=[]#final list that is used to insert into bt
                    dlst=[]
                    plst=Word_Tokenize(Clean_Text(new_name))#need to go through dig process below

                    for i in range (len(plst)):
                        if plst[i].isnumeric():
                            dig=True
                            converted=num2words(plst[i])
                            if '-' in converted:
                                converted_lst=converted.split('-')
                                for digword in converted_lst:
                                    dlst.append(digword)
                            else:
                                converted_lst=converted.split()
                                for digword in converted_lst:
                                    dlst.append(digword)
                            lst.append(plst[i])
                            continue
                        else:
                            dig=False
                            lst.append(plst[i])
                            dlst.append(plst[i])

                    self.insert(lst,cat,elem)

                    if dig:
                        #print(dlst)
                        self.insert(dlst,cat,elem)

                    if '-' in lst:#chain rest: xxx - xxx
                        base_entity=' '.join(lst[0:lst.index('-')])
                        lst.remove('-')#we won't have these types of punc in logs. remove because will have dl matching errors since score('-','.')=1 which passes
                        if base_entity not in chain_entities:
                            chain_entities[base_entity]=[cat,str(elem)]#need to change so use list instead of tuple
                        else:
                            chain_entities[base_entity][1]+='|'+str(elem)

                    if 'and' in lst and 'bar' in lst:#de luca cucina and bar
                        partial=lst[0:lst.index('and')]
                        self.insert(partial,cat,elem)

                    for i in range(len(lst)):#& <=> and
                        if lst[i]=='&':
                            lst[i]='and'
                            self.insert(lst,cat,elem)
                        elif lst[i]=='and':
                            lst[i]='&'
                            self.insert(lst,cat,elem)

                    if ',' in lst:#hotel vitale, ...
                        partial=lst[0:lst.index(',')]
                        self.insert(partial,cat,elem)
                        for i in range(len(partial)):#& <=> and
                            if partial[i]=='&':
                                partial[i]='and'
                                self.insert(partial,cat,elem)
                            elif partial[i]=='and':
                                partial[i]='&'
                                self.insert(partial,cat,elem)

                    if len(lst[0])==2:#sw hotel <-> s.w. hotel
                        acron=lst.pop(0)#treated as acronym
                        lst.insert(0,'.')
                        lst.insert(0,acron[1])
                        lst.insert(0,'.')
                        lst.insert(0,acron[0])
                        self.insert(lst,cat,elem)
                        for i in range(len(lst)):#& <=> and
                            if lst[i]=='&':
                                lst[i]='and'
                                self.insert(lst,cat,elem)
                            elif lst[i]=='and':
                                lst[i]='&'
                                self.insert(lst,cat,elem)
                        
        for entity in chain_entities:
            entity_list=entity.split()
            entity_info=chain_entities[entity]
            if '|' not in entity_info[1]:
                continue
            self.insert(entity_list,entity_info[0],entity_info[1])
            for i in range(len(entity_list)):#& <=> and
                if entity_list[i]=='&':
                    entity_list[i]='and'
                    self.insert(entity_list,entity_info[0],entity_info[1])
                elif entity_list[i]=='and':
                    entity_list[i]='&'
                    self.insert(entity_list,entity_info[0],entity_info[1])

    def initialize_w_ref(self,dic):#add db_dic if-/>
        #extract ref knowledge from db_dic TBDDDDDDDDDD
        #start of initialization
        dig=False
        chain_entities={}#keeping track of chain restaurants' ids i.e. {'entity name':('cat','id1|id2|id3|...')}
        for cat in dic.keys():
            for elem in dic[cat]:
                record = dic[cat][elem]

                if ('name' in record and record['name'] is not None):
                    new_name=record['name'].lower().strip()#cleans up name
                    if new_name == cat:
                        print('exceptional entity---------------------')
                        continue
                    lst=[]#final list that is used to insert into bt
                    dlst=[]
                    alt_dlst=[]
                    no_punc_lst=[]
                    plst=Word_Tokenize(Clean_Text(new_name))
                    #p=re.compile(r"^(\w+)(\'\w+)$")

                    for i in range (len(plst)):
                        if plst[i].isnumeric():
                            dig=True
                            for alt_dig in plst[i]:
                                alt_dlst.append(simple_num2words(int(alt_dig)))

                            converted=num2words(plst[i])
                            if '-' in converted:
                                converted_lst=converted.split('-')
                                for digword in converted_lst:
                                    dlst.append(digword)
                            else:
                                converted_lst=converted.split()
                                for digword in converted_lst:
                                    dlst.append(digword)
                            lst.append(plst[i])
                            continue
                        else:
                            lst.append(plst[i])
                            dlst.append(plst[i])
                            alt_dlst.append(plst[i])
                    
                    self.restricted_insert(lst,cat,elem)
                    #self.restricted_insert(no_punc_lst,cat,elem)#'alpha-milton guest house'

                    if dig:
                        self.restricted_insert(dlst,cat,elem)
                        self.restricted_insert(alt_dlst,cat,elem)

                    if '-' in lst:#chain rest: xxx - xxx
                        base_entity=' '.join(lst[0:lst.index('-')])
                        lst.remove('-')#we won't have these types of punc in logs. remove because will have dl matching errors since score('-','.')=1 which passes
                        self.restricted_insert(lst,cat,elem)
                        if base_entity not in chain_entities:
                            chain_entities[base_entity]=[cat,str(elem)]#need to change so use list instead of tuple
                        else:
                            chain_entities[base_entity][1]+='|'+str(elem)
                    
                    if 'and' in lst and 'bar' in lst:#de luca cucina and bar
                        partial=lst[0:lst.index('and')]
                        self.insert(partial,cat,elem)
                        
                    for i in range(len(lst)):#& <=> and
                        if lst[i]=='&':
                            lst[i]='and'
                            self.restricted_insert(lst,cat,elem)
                        elif lst[i]=='and':
                            lst[i]='&'
                            self.restricted_insert(lst,cat,elem)

                    if ',' in lst:#hotel vitale, ...
                        partial=lst[0:lst.index(',')]
                        self.restricted_insert(partial,cat,elem)
                        for i in range(len(partial)):#& <=> and
                            if partial[i]=='&':
                                partial[i]='and'
                                self.restricted_insert(partial,cat,elem)
                            elif partial[i]=='and':
                                partial[i]='&'
                                self.restricted_insert(partial,cat,elem)

                    if len(lst[0])==2:#sw hotel <-> s.w. hotel
                        acron=lst.pop(0)#treated as acronym
                        lst.insert(0,acron[1]+'.')
                        lst.insert(0,acron[0]+'.')
                        self.restricted_insert(lst,cat,elem)
                        for i in range(len(lst)):#& <=> and
                            if lst[i]=='&':
                                lst[i]='and'
                                self.restricted_insert(lst,cat,elem)
                            elif lst[i]=='and':
                                lst[i]='&'
                                self.restricted_insert(lst,cat,elem)
                        
        for entity in chain_entities:
            entity_list=entity.split()
            entity_info=chain_entities[entity]
            if '|' not in entity_info[1]:
                continue
            self.restricted_insert(entity_list,entity_info[0],entity_info[1])
            for i in range(len(entity_list)):#& <=> and
                if entity_list[i]=='&':
                    entity_list[i]='and'
                    self.restricted_insert(entity_list,entity_info[0],entity_info[1])
                elif entity_list[i]=='and':
                    entity_list[i]='&'
                    self.restricted_insert(entity_list,entity_info[0],entity_info[1])
    
    def sp_initialize(self,spdic):
        dig=False
        chain_entities={}
        #only for db.json in dstc10_track2_task1
        for cat in spdic.keys():
            for entry in spdic[cat]:
                if (entry['id'] is not None) and (entry['name'] is not None):
                    new_name=entry['name'].lower().strip()#cleans up name
                    if new_name == cat:
                        print('exceptional entity---------------------')
                        continue
                    lst=[]#final list that is used to insert into bt
                    dlst=[]
                    plst=Word_Tokenize(Clean_Text(new_name))#need to go through dig process below

                    for i in range (len(plst)):
                        if plst[i].isnumeric():
                            dig=True
                            converted=num2words(plst[i])
                            if '-' in converted:
                                converted_lst=converted.split('-')
                                for digword in converted_lst:
                                    dlst.append(digword)
                            else:
                                converted_lst=converted.split()
                                for digword in converted_lst:
                                    dlst.append(digword)
                            lst.append(plst[i])
                            continue
                        else:
                            dig=False
                            lst.append(plst[i])
                            dlst.append(plst[i])

                    self.insert(lst,cat,entry['id'])

                    if dig:
                        print(dlst)
                        self.insert(dlst,cat,entry['id'])

                    if '-' in lst:#chain rest: xxx - xxx
                        base_entity=' '.join(lst[0:lst.index('-')])
                        lst.remove('-')#we won't have these types of punc in logs. remove because will have dl matching errors since score('-','.')=1 which passes
                        if base_entity not in chain_entities:
                            chain_entities[base_entity]=[cat,str(entry['id'])]#need to change so use list instead of tuple
                        else:
                            chain_entities[base_entity][1]+='|'+str(entry['id'])

                    if 'and' in lst and 'bar' in lst:#de luca cucina and bar
                        partial=lst[0:lst.index('and')]
                        self.insert(partial,cat,entry['id'])

                    for i in range(len(lst)):#& <=> and
                        if lst[i]=='&':
                            lst[i]='and'
                            self.insert(lst,cat,entry['id'])
                        elif lst[i]=='and':
                            lst[i]='&'
                            self.insert(lst,cat,entry['id'])

                    if ',' in lst:#hotel vitale, ...
                        partial=lst[0:lst.index(',')]
                        self.insert(partial,cat,entry['id'])
                        for i in range(len(partial)):#& <=> and
                            if partial[i]=='&':
                                partial[i]='and'
                                self.insert(partial,cat,entry['id'])
                            elif partial[i]=='and':
                                partial[i]='&'
                                self.insert(partial,cat,entry['id'])

                    if len(lst[0])==2:#sw hotel <-> s.w. hotel
                        acron=lst.pop(0)#treated as acronym
                        lst.insert(0,'.')
                        lst.insert(0,acron[1])
                        lst.insert(0,'.')
                        lst.insert(0,acron[0])
                        self.insert(lst,cat,entry['id'])
                        for i in range(len(lst)):#& <=> and
                            if lst[i]=='&':
                                lst[i]='and'
                                self.insert(lst,cat,entry['id'])
                            elif lst[i]=='and':
                                lst[i]='&'
                                self.insert(lst,cat,entry['id'])
                        
        for entity in chain_entities:
            entity_list=entity.split()
            entity_info=chain_entities[entity]
            if '|' not in entity_info[1]:
                continue
            self.insert(entity_list,entity_info[0],entity_info[1])
            for i in range(len(entity_list)):#& <=> and
                if entity_list[i]=='&':
                    entity_list[i]='and'
                    self.insert(entity_list,entity_info[0],entity_info[1])
                elif entity_list[i]=='and':
                    entity_list[i]='&'
                    self.insert(entity_list,entity_info[0],entity_info[1])

    def initialize_multiwoz(self):
        dic_lst = ['/content/drive/MyDrive/Research/ner/Multiwoz/multiwoz/db/attraction_db.json',
                   '',
                   '',
                   '',
                   '',
                   '',
                   '',
                   
        ]
        for f in dic_lst:
            print('Starting ', f)
            db = open(f,'r')
            dic = json.load(db)
            db.close()
            dig=False
            chain_entities={}#keeping track of chain restaurants' ids i.e. {'entity name':('cat','id1|id2|id3|...')}
            for record in dic:            
                if ('name' in record and record['name'] is not None and 'type' in record and record['type'] is not None):
                    new_name=record['name'].lower().strip()#cleans up name
                    if 'id' in record and record['id'] is not None:
                        elem = record['id']
                    else:
                        elem = 'xxx'
                    if new_name == cat:
                        print('exceptional entity---------------------')
                        continue
                    lst=[]#final list that is used to insert into bt
                    dlst=[]
                    alt_dlst=[]
                    no_punc_lst=[]
                    plst=Word_Tokenize(Clean_Text(new_name))
                    #p=re.compile(r"^(\w+)(\'\w+)$")
    
                    for i in range (len(plst)):
                        if plst[i].isnumeric():
                            dig=True
                            for alt_dig in plst[i]:
                                alt_dlst.append(simple_num2words(int(alt_dig)))
    
                            converted=num2words(plst[i])
                            if '-' in converted:
                                converted_lst=converted.split('-')
                                for digword in converted_lst:
                                    dlst.append(digword)
                            else:
                                converted_lst=converted.split()
                                for digword in converted_lst:
                                    dlst.append(digword)
                            lst.append(plst[i])
                            continue
                        else:
                            lst.append(plst[i])
                            dlst.append(plst[i])
                            alt_dlst.append(plst[i])
                    
                    for w in lst:
                        if w.isalnum():
                            no_punc_lst.append(w)
                    
                    self.restricted_insert(lst,cat,elem)
                    self.restricted_insert(no_punc_lst,cat,elem)
    
                    if dig:
                        self.restricted_insert(dlst,cat,elem)
                        self.restricted_insert(alt_dlst,cat,elem)
    
                    if '-' in lst:#chain rest: xxx - xxx
                        base_entity=' '.join(lst[0:lst.index('-')])
                        lst.remove('-')#we won't have these types of punc in logs. remove because will have dl matching errors since score('-','.')=1 which passes
                        self.restricted_insert(lst,cat,elem)
                        if base_entity not in chain_entities:
                            chain_entities[base_entity]=[cat,str(elem)]#need to change so use list instead of tuple
                        else:
                            chain_entities[base_entity][1]+='|'+str(elem)
                    
                    if 'and' in lst and 'bar' in lst:#de luca cucina and bar
                        partial=lst[0:lst.index('and')]
                        self.insert(partial,cat,elem)
                        
                    for i in range(len(lst)):#& <=> and
                        if lst[i]=='&':
                            lst[i]='and'
                            self.restricted_insert(lst,cat,elem)
                        elif lst[i]=='and':
                            lst[i]='&'
                            self.restricted_insert(lst,cat,elem)
    
                    if ',' in lst:#hotel vitale, ...
                        partial=lst[0:lst.index(',')]
                        self.restricted_insert(partial,cat,elem)
                        for i in range(len(partial)):#& <=> and
                            if partial[i]=='&':
                                partial[i]='and'
                                self.restricted_insert(partial,cat,elem)
                            elif partial[i]=='and':
                                partial[i]='&'
                                self.restricted_insert(partial,cat,elem)
    
                    if len(lst[0])==2:#sw hotel <-> s.w. hotel
                        acron=lst.pop(0)#treated as acronym
                        lst.insert(0,acron[1]+'.')
                        lst.insert(0,acron[0]+'.')
                        self.restricted_insert(lst,cat,elem)
                        for i in range(len(lst)):#& <=> and
                            if lst[i]=='&':
                                lst[i]='and'
                                self.restricted_insert(lst,cat,elem)
                            elif lst[i]=='and':
                                lst[i]='&'
                                self.restricted_insert(lst,cat,elem)
                        
        for entity in chain_entities:
            entity_list=entity.split()
            entity_info=chain_entities[entity]
            if '|' not in entity_info[1]:
                continue
            self.restricted_insert(entity_list,entity_info[0],entity_info[1])
            for i in range(len(entity_list)):#& <=> and
                if entity_list[i]=='&':
                    entity_list[i]='and'
                    self.restricted_insert(entity_list,entity_info[0],entity_info[1])
                elif entity_list[i]=='and':
                    entity_list[i]='&'
                    self.restricted_insert(entity_list,entity_info[0],entity_info[1])
def main():
    kb=open('/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data_eval/knowledge.json','r')
    dic=json.load(kb)

    bt=BabyTrie()
    bt.initialize_w_ref(dic)
    kb.close()
    res=bt.isinTrie("ram's hotel")
    #print(res[0],res[1],res[2],sep='\n')
    print(res)

if __name__=='__main__':
    main()


ModuleNotFoundError: ignored

In [ ]:
from BabyTrie_DSTC10_v3 import BabyTrie
import json
kb=open('/content/drive/MyDrive/Research/ner/huggingface/alexa-with-dstc9-track1-dataset/data_eval/knowledge.json','r')
dic=json.load(kb)

bt=BabyTrie()

bt.initialize_w_ref(dic)
test="nineteen six mission"
res=bt.isinTrie(test)
print(res)

(['nineteen', 'six', 'mission'], {})


In [ ]:
#@title PREPROCESS
%%writefile preprocess.py
import json
import re
import argparse
from BabyTrie import BabyTrie
from argparse import Namespace

def get_bt(dic):
    '''
    Given a dictionary with categories of entities in the form:
    {'some category':{'0':{'name':'some name'},'1':{'name':'some other name'},...},'some other category':{'0':...},...}
    Return a BabyTrie that contains all entities and the leaf nodes contianing a list of their corresponding categories
    
    Note: A large portion of this code under the if statement is cleaning up the list of words within the name phrase
    -------------------------------------------------------------------------------------------------------------------
    Example:
    dic={'restaurant':{'0':{'name':'Jade Garden'}}}
    bt=get_bt(dic)#the name 'jade' and 'garden' are created as nodes and inserted into the BabyTrie
    
    '''
    bt=BabyTrie()
    for cat in dic.keys():
        for elem in dic[cat]:
            record = dic[cat][elem]

            if ('name' in record and record['name'] is not None):
                new_name=record['name'].lower().strip()#cleans up name
                if new_name == cat:
                    print('exceptional entity---------------------')
                    continue
                lst=[]#final list that is used to insert into bt
                plst=re.findall(r"[\w']+|[&.,!?;]", new_name)#premature list, can have ex. "McDonald's" instead of "McDonald" and "'s"
                p=re.compile(r"^(\w+)(\'\w+)$")

                for i in range (len(plst)):
                    if p.match(plst[i]):
                        new_word1,new_word2=p.match(plst[i]).group(1),p.match(plst[i]).group(2)
                        lst.append(new_word1)
                        lst.append(new_word2)
                    else:
                        lst.append(plst[i])
                bt.insert(lst,cat)
                for i in range(len(lst)):#& <=> and
                    if lst[i]=='&':
                        lst[i]='and'
                        bt.insert(lst,cat)
                    elif lst[i]=='and':
                        lst[i]='&'
                        bt.insert(lst,cat)
    return bt

def extraction_of_log1(lst):#in this case our log1 file (for train.txt) is a list of dictionaries
    res={}
    id=0
    for i in range(0,len(lst)):
        for elem in lst[i]:
            if elem['text'] not in res:
                res[elem['text']]=id
                id+=1
    return res

def extraction_of_log2(dic):
    res={}
    id=0
    for cat in dic.keys():
        for elem in dic[cat]:
            for dialogue_num in dic[cat][elem]['docs']:
                num1,num2=1,2
                if dic[cat][elem]['docs'][dialogue_num]['title'] not in res:
                    res[dic[cat][elem]['docs'][dialogue_num]['title']]=(id,num1)
                if dic[cat][elem]['docs'][dialogue_num]['body'] not in res:
                    res[dic[cat][elem]['docs'][dialogue_num]['body']]=(id,num2)
    return res

def printres(ofstream,res):
    '''
    Given a result tuple that is generated from the isinTrie method in the class of BabyTrie,
    Outputs the result in two columns containing labels in a file (tab in between)
    Note: the labeling of 'B_cat','I_cat', and 'O' is used.
    B_cat = begining of an entity of a category
    I_cat = intermediate (in the middle) of an entity of a category
    O = not an entity
    ----------------------------------------------------------------------------------------
    Example: (same output from isinTrie example)
    res=(['i','went','to','jade','garden','last','night','.'], {'restaurant':(3,4)})
                        lst                                            dic
    printres(res)

    Output:
    i       O
    went    O
    to      O
    jade    B-restaurant
    garden  I-restaurant
    last    O
    night   O
    .       O
    '''
    lst=res[0]
    dic=res[1]
    sen_label=[[word,'O'] for word in lst]
    for cat in dic.keys():
        for item in dic[cat]:
            start,end=item[0],item[1]
            first=True
            for i in range(start,end+1):
                if (sen_label[i][-1]!='O'):
                    if first:
                        record='|B-'+cat
                        first=False
                    else:
                        record='|I-'+cat
                    sen_label[i][-1]+=record
                else:
                    if first:
                        sen_label[i][-1]='B-'+cat
                        first=False
                    else:
                        sen_label[i][-1]='I-'+cat
    for elem in sen_label:
        ofstream.write('{}\t{}\n'.format(elem[0],elem[1]))
    ofstream.write('\n')

def printres_ne(ofstream,res):
    '''
    
    '''
    lst=res[0]
    dic=res[1]
    sen_label=[[word,'O'] for word in lst]
    if len(dic)==0:#if no entity in dic, do not output (testing)
        return
    for cat in dic.keys():
        for item in dic[cat]:
            start,end=item[0],item[1]
            first=True
            for i in range(start,end+1):
                if (sen_label[i][-1]!='O'):
                    if first:
                        record='|B-'+cat
                        first=False
                    else:
                        record='|I-'+cat
                    sen_label[i][-1]+=record
                else:
                    if first:
                        sen_label[i][-1]='B-'+cat
                        first=False
                    else:
                        sen_label[i][-1]='I-'+cat
    for elem in sen_label:
        ofstream.write('{}\t{}\n'.format(elem[0],elem[1]))
    ofstream.write('\n')

def printres_o(ofstream,res):
    '''
    
    '''
    lst=res[0]
    dic=res[1]
    sen_label=[[word,'O'] for word in lst]
    if len(dic)!=0:#if entity in dic, do not output (testing)
        return
    if 'attraction' not in dic:
        return
    for cat in dic.keys():
        for item in dic[cat]:
            start,end=item[0],item[1]
            first=True
            for i in range(start,end+1):
                if (sen_label[i][-1]!='O'):
                    if first:
                        record='|B-'+cat
                        first=False
                    else:
                        record='|I-'+cat
                    sen_label[i][-1]+=record
                else:
                    if first:
                        sen_label[i][-1]='B-'+cat
                        first=False
                    else:
                        sen_label[i][-1]='I-'+cat
    for elem in sen_label:
        ofstream.write('{}\t{}\n'.format(elem[0],elem[1]))
    ofstream.write('\n')

def main():

    parser=argparse.ArgumentParser(description='NER preprocess dataset')
    parser.add_argument("--log_file",type=str,default='logs.json',help="input log file in json")
    parser.add_argument("--knowledge_file",type=str,default='knowledge.json',help="input knowledge file in json")
    parser.add_argument("--output_file",type=str,default='output.txt',help="output file for NER training")
    parser.add_argument("--extract_lognum",type=int,default=1,help="1 for log (default), 2 for log included in knowledge base")
    parser.add_argument("--printres_type",type=int,default=1,help="1 for printres (default), 2 for printres_ne (only named entity sentences, 3 for printres_o (only non-named entities")
    args = parser.parse_args()
    
    #opens files first to load
    logs1=open(args.log_file,'r')
    data1=json.load(logs1)
    k=open(args.knowledge_file,'r')
    dic=json.load(k)
    

    bt=get_bt(dic)#setting up bt based on knowledge base
    if args.extract_lognum==1:
        dset=extraction_of_log1(data1)#extract logs1
    elif args.extract_lognum==2:
        dset=extraction_of_log2(dic)#extract additional data from knowledge base
    
    outFile=open(args.output_file,'w')
    for senkey in dset:#write results of log to file
        res=bt.isinTrie(senkey)
        if args.printres_type==1:
            printres(outFile,res)
        elif args.printres_type==2:
            printres_ne(outFile,res)
        elif args.printres_type==3:    
            printres_o(outFile,res)

    logs1.close()
    k.close()
    outFile.close()
    
if __name__ == "__main__":
    main()


Overwriting preprocess.py


In [ ]:
#process logs with knowledge base and label in two column format
!python preprocess.py --output_file "test_ne_other.txt" --printres_type 2 --knowledge_file '/content/drive/MyDrive/Research/huggingface/alexa-with-dstc9-track1-dataset/data_eval/knowledge.json' --log_file '/content/drive/MyDrive/Research/huggingface/alexa-with-dstc9-track1-dataset/data_eval/test/logs.json'

# Datascript/Datasets

In [ ]:
%env HOME=/content/drive/MyDrive/
#!mkdir -p ~/Research/huggingface
%cd ~/Research/ner

env: HOME=/content/drive/MyDrive/
/content/drive/MyDrive/Research/ner


In [ ]:
#to see any source code (ctrl+click on link)
import datasets
print(datasets.__path__)
print(datasets.__version__)
!ls -l /usr/local/lib/python3.6/dist-packages/datasets/

In [ ]:
#@title TEXT_CLASS DATASCRIPT
%%writefile text_class.py
import logging
import json
import datasets
#DOESNT WORK. Run on hpc

_CITATION = ''

_DESCRIPTION = ''
_URL = ""
#these three need to be modified accordingly
_TRAINING_FILE = 'classifier_examples_train.json'
_DEV_FILE = 'classifier_examples_test.json'


class text_classConfig(datasets.BuilderConfig):
    """BuilderConfig for text_class"""

    def __init__(self, **kwargs):
        """BuilderConfig for text_class.
        Args:
          **kwargs: keyword arguments forwarded to super.
        """
        super(text_classConfig, self).__init__(**kwargs)


class text_class2021(datasets.GeneratorBasedBuilder):
    """text_class dataset."""

    BUILDER_CONFIGS = [
        text_classConfig(name="text_class", version=datasets.Version("1.0.0"), description="text classification dataset for dstc10 track2"),
    ]

    def _info(self):
        return datasets.DatasetInfo(
            description=_DESCRIPTION,
            features=datasets.Features(
                {
                    "id": datasets.Value("string"),
                    "title": datasets.Sequence(datasets.Value("string")),
                    "body": datasets.Sequence(datasets.Value("string")),
                    "label": datasets.Sequence(
                        datasets.features.ClassLabel(
                            names=[
                                "true",
                                "false",
                            ]
                        )
                    ),
                }
            ),
            supervised_keys=None,
            homepage="N/A",
            citation=_CITATION,
        )
    
    def _split_generators(self, dl_manager):
        """Returns SplitGenerators."""
        urls_to_download = {
            "train": f"{_URL}{_TRAINING_FILE}",
            "dev": f"{_URL}{_DEV_FILE}",
        }
        downloaded_files = dl_manager.download_and_extract(urls_to_download)

        return [
            datasets.SplitGenerator(name=datasets.Split.TRAIN, gen_kwargs={"filepath": downloaded_files["train"]}),
            datasets.SplitGenerator(name=datasets.Split.VALIDATION, gen_kwargs={"filepath": downloaded_files["dev"]}),
        ]
    
    def _generate_examples(self, filepath):
        logging.info("⏳ Generating examples from = %s", filepath)
        with open(filepath, encoding="utf-8") as f:
            dic=json.load(f)
            guid = 0
            title = []
            body = []
            labels = []
            for entry in dic:
                yield guid,{
                    "id": str(guid),
                    "title":entry['title'],
                    'body':entry['body'],
                    'label':entry['label'],
                }
                guid+=1

Overwriting text_class.py


In [ ]:
#@title SIMPLER_NER2021 DATASCRIPT
%%writefile simple_ner2021.py
import logging

import datasets


_CITATION = ''

_DESCRIPTION = ''
_URL = ""
#these three need to be modified accordingly
_TRAINING_FILE = 'train_ne.txt'
_DEV_FILE = 'valid_ne.txt'
_TEST_FILE = 'train_o.txt'


class simple_ner2021Config(datasets.BuilderConfig):
    """BuilderConfig for simple_ner2021"""

    def __init__(self, **kwargs):
        """BuilderConfig forsimple_ner2021.
        Args:
          **kwargs: keyword arguments forwarded to super.
        """
        super(simple_ner2021Config, self).__init__(**kwargs)


class simple_ner2021(datasets.GeneratorBasedBuilder):
    """simple_ner2021 dataset."""

    BUILDER_CONFIGS = [
        simple_ner2021Config(name="simple_ner2021", version=datasets.Version("1.0.0"), description="simple named entity extraction (2021) dataset"),
    ]

    def _info(self):
        return datasets.DatasetInfo(
            description=_DESCRIPTION,
            features=datasets.Features(
                {
                    "id": datasets.Value("string"),
                    "tokens": datasets.Sequence(datasets.Value("string")),
                    "ner_tags": datasets.Sequence(
                        datasets.features.ClassLabel(
                            names=[
                                "O",
                                "B-hotel",
                                "I-hotel",
                                "B-restaurant",
                                "I-restaurant",
                                "B-attraction",
                                "I-attraction",
                            ]
                        )
                    ),
                }
            ),
            supervised_keys=None,
            homepage="N/A",
            citation=_CITATION,
        )
    
    def _split_generators(self, dl_manager):
        """Returns SplitGenerators."""
        urls_to_download = {
            "train": f"{_URL}{_TRAINING_FILE}",
            "dev": f"{_URL}{_DEV_FILE}",
            "test": f"{_URL}{_TEST_FILE}",
        }
        downloaded_files = dl_manager.download_and_extract(urls_to_download)

        return [
            datasets.SplitGenerator(name=datasets.Split.TRAIN, gen_kwargs={"filepath": downloaded_files["train"]}),
            datasets.SplitGenerator(name=datasets.Split.VALIDATION, gen_kwargs={"filepath": downloaded_files["dev"]}),
            datasets.SplitGenerator(name=datasets.Split.TEST, gen_kwargs={"filepath": downloaded_files["test"]}),
        ]
    
    def _generate_examples(self, filepath):
        logging.info("⏳ Generating examples from = %s", filepath)
        with open(filepath, encoding="utf-8") as f:
            guid = 0
            tokens = []
            ner_tags = []
            for line in f:
                if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                    if tokens:
                        yield guid, {
                            "id": str(guid),
                            "tokens": tokens,
                            "ner_tags": ner_tags,
                        }
                        guid += 1
                        tokens = []
                        ner_tags = []
                else:
                    # simple_ner2021 tokens are tab separated
                    splits = line.split("\t")
                    tokens.append(splits[0])
                    ner_tags.append(splits[1].rstrip())
            # last example
            yield guid, {
                "id": str(guid),
                "tokens": tokens,
                "ner_tags": ner_tags,
            }


In [ ]:
#%cd /root/.cache/huggingface/datasets/simple_ner2021
%cd /root/.cache/huggingface/datasets/text_class
#makes sure dataset is not cached before, don't want to use wrong dataset
#!ls
%cd ~/Research/ner

#/content/drive/MyDrive/Research/ner/text_classifier/classifier_examples_train.json
#/content/drive/MyDrive/Research/ner/text_classifier/classifier_examples_test.json

[Errno 2] No such file or directory: '/root/.cache/huggingface/datasets/text_class'
/content/drive/MyDrive/Research/ner/text_classifier
/content/drive/MyDrive/Research/ner


In [ ]:
#load dataset for ner training model
from datasets import load_dataset
dataset = load_dataset('text_class.py', data_files={'train': 'classifier_examples_train.json','validation': 'classifier_examples_test.json'})
dataset['train']#to check with below
'''
dataset = load_dataset('text_class.py', data_files={'train': 'train.txt',
                                                        'validation': 'valid.txt',
                                                        'test':'test.txt'})
'''

In [ ]:
#%cd ~/Research/huggingface/transformers/examples/token-classification
%cd ~/Research/ner

/content/drive/MyDrive/Research/ner


In [ ]:
#checks with the above dataset. The number here should be 1 less than the num_rows inside that element in the dataset
!egrep "^$" test_ne_other.txt | wc -l

In [ ]:
#@title TEST_RUN_NER
%%writefile test_run_ner.py
# coding=utf-8
# Copyright 2020 The HuggingFace Team All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
Fine-tuning the library models for token classification.
"""
# You can also adapt this script on your own token classification task and datasets. Pointers for this are left as
# comments.

import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import numpy as np
from datasets import ClassLabel, load_dataset
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score

import transformers
from transformers import (
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    HfArgumentParser,
    PreTrainedTokenizerFast,
    Trainer,
    TrainingArguments,
    set_seed,
)
from transformers.trainer_utils import is_main_process


logger = logging.getLogger(__name__)


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    task_name: Optional[str] = field(default="ner", metadata={"help": "The name of the task (ner, pos...)."})
    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    train_file: Optional[str] = field(
        default=None, metadata={"help": "The input training data file (a csv or JSON file)."}
    )
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file to evaluate on (a csv or JSON file)."},
    )
    test_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input test data file to predict on (a csv or JSON file)."},
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    pad_to_max_length: bool = field(
        default=False,
        metadata={
            "help": "Whether to pad all samples to model maximum sentence length. "
            "If False, will pad the samples dynamically when batching to the maximum length in the batch. More "
            "efficient on GPU but very bad for TPU."
        },
    )
    label_all_tokens: bool = field(
        default=False,
        metadata={
            "help": "Whether to put the label for one word on all tokens of generated by that word or just on the "
            "one (in which case the other tokens will have a padding index)."
        },
    )
    datascript_file: Optional[str] = field(
        default=None, metadata={"help": "customize script to load dataset"}
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in ["txt", "csv", "json"], "`train_file` should be a csv or a json file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in ["txt", "csv", "json"], "`validation_file` should be a csv or a json file."
        self.task_name = self.task_name.lower()


def main():
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
    if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
        # If we pass only one argument to the script and it's the path to a json file,
        # let's parse it to get our arguments.
        model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
    else:
        model_args, data_args, training_args = parser.parse_args_into_dataclasses()

    if (
        os.path.exists(training_args.output_dir)
        and os.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty."
            "Use --overwrite_output_dir to overcome."
        )

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if is_main_process(training_args.local_rank) else logging.WARN,
    )

    # Log on each process the small summary:
    logger.warning(
        f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
        + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
    )
    # Set the verbosity to info of the Transformers logger (on main process only):
    if is_main_process(training_args.local_rank):
        transformers.utils.logging.set_verbosity_info()
        transformers.utils.logging.enable_default_handler()
        transformers.utils.logging.enable_explicit_format()
    logger.info("Training/evaluation parameters %s", training_args)

    # Set seed before initializing model.
    set_seed(training_args.seed)

    # Get the datasets: you can either provide your own CSV/JSON/TXT training and evaluation files (see below)
    # or just provide the name of one of the public datasets available on the hub at https://huggingface.co/datasets/
    # (the dataset will be downloaded automatically from the datasets Hub).
    #
    # For CSV/JSON files, this script will use the column called 'text' or the first column if no column called
    # 'text' is found. You can easily tweak this behavior (see below).
    #
    # In distributed training, the load_dataset function guarantee that only one local process can concurrently
    # download the dataset.
    if data_args.dataset_name is not None:
        # Downloading and loading a dataset from the hub.
        datasets = load_dataset(data_args.dataset_name, data_args.dataset_config_name)
    else:
        data_files = {}
        if data_args.train_file is not None:
            data_files["train"] = data_args.train_file
        if data_args.validation_file is not None:
            data_files["validation"] = data_args.validation_file
        if data_args.test_file is not None:
            data_files["test"] = data_args.test_file
        if data_args.datascript_file is not None:
            extension=data_args.datascript_file
        else:
            extension = data_args.train_file.split(".")[-1]
        datasets = load_dataset(extension, data_files=data_files)
    # See more about loading any type of standard or custom dataset (from files, python dict, pandas DataFrame, etc) at
    # https://huggingface.co/docs/datasets/loading_datasets.html.

    if training_args.do_train:
        column_names = datasets["train"].column_names
        features = datasets["train"].features
    else:
        column_names = datasets["validation"].column_names
        features = datasets["validation"].features
    text_column_name = "tokens" if "tokens" in column_names else column_names[0]
    label_column_name = (
        f"{data_args.task_name}_tags" if f"{data_args.task_name}_tags" in column_names else column_names[1]
    )

    # In the event the labels are not a `Sequence[ClassLabel]`, we will need to go through the dataset to get the
    # unique labels.
    def get_label_list(labels):
        unique_labels = set()
        for label in labels:
            unique_labels = unique_labels | set(label)
        label_list = list(unique_labels)
        label_list.sort()
        return label_list

    if isinstance(features[label_column_name].feature, ClassLabel):
        label_list = features[label_column_name].feature.names
        # No need to convert the labels since they are already ints.
        label_to_id = {i: i for i in range(len(label_list))}
    else:
        label_list = get_label_list(datasets["train"][label_column_name])
        label_to_id = {l: i for i, l in enumerate(label_list)}
    num_labels = len(label_list)

    # Load pretrained model and tokenizer
    #
    # Distributed training:
    # The .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.
    config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        num_labels=num_labels,
        finetuning_task=data_args.task_name,
        cache_dir=model_args.cache_dir,
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
        use_fast=True,
    )
    model = AutoModelForTokenClassification.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
    )

    # Tokenizer check: this script requires a fast tokenizer.
    if not isinstance(tokenizer, PreTrainedTokenizerFast):
        raise ValueError(
            "This example script only works for models that have a fast tokenizer. Checkout the big table of models "
            "at https://huggingface.co/transformers/index.html#bigtable to find the model types that meet this "
            "requirement"
        )

    # Preprocessing the dataset
    # Padding strategy
    padding = "max_length" if data_args.pad_to_max_length else False

    # Tokenize all texts and align the labels with them.
    def tokenize_and_align_labels(examples):
        tokenized_inputs = tokenizer(
            examples[text_column_name],
            padding=padding,
            truncation=True,
            # We use this argument because the texts in our dataset are lists of words (with a label for each word).
            is_split_into_words=True,
        )
        labels = []
        for i, label in enumerate(examples[label_column_name]):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            previous_word_idx = None
            label_ids = []
            for word_idx in word_ids:
                # Special tokens have a word id that is None. We set the label to -100 so they are automatically
                # ignored in the loss function.
                if word_idx is None:
                    label_ids.append(-100)
                # We set the label for the first token of each word.
                elif word_idx != previous_word_idx:
                    label_ids.append(label_to_id[label[word_idx]])
                # For the other tokens in a word, we set the label to either the current label or -100, depending on
                # the label_all_tokens flag.
                else:
                    label_ids.append(label_to_id[label[word_idx]] if data_args.label_all_tokens else -100)
                previous_word_idx = word_idx

            labels.append(label_ids)
        tokenized_inputs["labels"] = labels
        return tokenized_inputs

    tokenized_datasets = datasets.map(
        tokenize_and_align_labels,
        batched=True,
        num_proc=data_args.preprocessing_num_workers,
        load_from_cache_file=not data_args.overwrite_cache,
    )

    # Data collator
    data_collator = DataCollatorForTokenClassification(tokenizer)

    # Metrics
    def compute_metrics(p):
        predictions, labels = p
        predictions = np.argmax(predictions, axis=2)

        # Remove ignored index (special tokens)
        true_predictions = [
            [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        true_labels = [
            [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]

        return {
            "accuracy_score": accuracy_score(true_labels, true_predictions),
            "precision": precision_score(true_labels, true_predictions),
            "recall": recall_score(true_labels, true_predictions),
            "f1": f1_score(true_labels, true_predictions),
        }

    # Initialize our Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"] if training_args.do_train else None,
        eval_dataset=tokenized_datasets["validation"] if training_args.do_eval else None,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    # Training
    if training_args.do_train:
        train_result = trainer.train(
            model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
        )
        trainer.save_model()  # Saves the tokenizer too for easy upload

        output_train_file = os.path.join(training_args.output_dir, "train_results.txt")
        if trainer.is_world_process_zero():
            with open(output_train_file, "w") as writer:
                logger.info("***** Train results *****")
                for key, value in sorted(train_result.metrics.items()):
                    logger.info(f"  {key} = {value}")
                    writer.write(f"{key} = {value}\n")

            # Need to save the state, since Trainer.save_model saves only the tokenizer with the model
            trainer.state.save_to_json(os.path.join(training_args.output_dir, "trainer_state.json"))

    # Evaluation
    results = {}
    if training_args.do_eval:
        logger.info("*** Evaluate ***")

        results = trainer.evaluate()

        output_eval_file = os.path.join(training_args.output_dir, "eval_results_ner.txt")
        if trainer.is_world_process_zero():
            with open(output_eval_file, "w") as writer:
                logger.info("***** Eval results *****")
                for key, value in results.items():
                    logger.info(f"  {key} = {value}")
                    writer.write(f"{key} = {value}\n")

    # Predict
    if training_args.do_predict:
        logger.info("*** Predict ***")

        test_dataset = tokenized_datasets["test"]#SELF COMMENT:always uses cached!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        predictions, labels, metrics = trainer.predict(test_dataset)
        predictions = np.argmax(predictions, axis=2)

        # Remove ignored index (special tokens)
        true_predictions = [
            [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]

        output_test_results_file = os.path.join(training_args.output_dir, "test_results.txt")
        if trainer.is_world_process_zero():
            with open(output_test_results_file, "w") as writer:
                for key, value in sorted(metrics.items()):
                    logger.info(f"  {key} = {value}")
                    writer.write(f"{key} = {value}\n")

        # Save predictions
        output_test_predictions_file = os.path.join(training_args.output_dir, "test_predictions.txt")
        if trainer.is_world_process_zero():
            with open(output_test_predictions_file, "w") as writer:
                for prediction in true_predictions:
                    writer.write(" ".join(prediction) + "\n")

    return results


def _mp_fn(index):
    # For xla_spawn (TPUs)
    main()


if __name__ == "__main__":
    main()

In [ ]:
#running model
'''
!python3 test_run_ner.py \
  --model_name_or_path bert-base-uncased \
  --datascript_file simple_ner2021.py \
  --output_dir ./output_ne \
  --train_file train_ne.txt \
  --validation_file valid_ne.txt \
  --test_file test_ne.txt \
  --do_train \
  --do_eval
'''
!python3 test_run_ner.py \
  --model_name_or_path M1 \
  --datascript_file simple_ner2021.py \
  --output_dir ./M1train_o_predictions \
  --train_file train_ne.txt \
  --validation_file valid_ne.txt \
  --test_file train_o.txt \
  --do_predict


In [ ]:
#@title PREDICTIONS OUTPUT CONVERTER
#checking and outputing the predictions from model to the 2 column format
def check(file1,file2):
  #checks if they can be matched
  counter1=0
  counter2=0
  for line in file1:
    if line=='\n':
      counter1+=1
  for line in file2:
    counter2+=1
  return counter1,counter2

def main():
  f1=open('train_o.txt','r')
  f2=open('/content/drive/MyDrive/Research/ner/M1train_o_predictions/test_predictions.txt','r')
  #x,y=check(f1,f2)
  #print(x+1,y)
  
  rf=open('M1train_predictions_o.txt','w')
  rlst_label=[]
  for line in f2:
    if line!='\n':
      label_lst=line.split()
      rlst_label.extend(label_lst)
  for line in f1:
    if line!='\n':
      word=line.split('\t')[0]
      label=rlst_label.pop(0)
      rf.write(word+'\t'+label+'\n')
    else:
      rf.write('\n')
  
if __name__ =="__main__":
  main()

In [ ]:
#@title TOKEN_PREDICT (TESTING)
#%%writefile token_predict.py
#other testing method for predictions (manual examples)
import torch
import numpy as np
import transformers
from transformers import (
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer,
)

class TokenClassifier:
   def __init__(self, model_path, use_fast=True, from_tf=False):
       self.config = AutoConfig.from_pretrained(model_path)
       self.tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=use_fast)
       self.model = AutoModelForTokenClassification.from_pretrained(model_path, config=self.config, from_tf=from_tf)

   def predict(self, query):
       # Evaluation is done using CPU here
       input_ids_cpu = self.tokenizer.encode(query)
       # add a dummy batch dimension
       input_ids = torch.tensor([input_ids_cpu])
       #print(input_ids.shape)

       with torch.no_grad():
           result = self.model(input_ids=input_ids)
           logits = result["logits"].detach().cpu().numpy()
           # reduce the last axis with class with maximum score
           token_predict = np.argmax(logits, axis=-1)
           return token_predict, logits

if __name__ == "__main__":
    # try a single sentence
    query = "yes	,	the	alexander	offers	those	.	el	shaddai	,	gonville	hotel	,	or	the	university	arms	hotel	all	do	as	well	,	if	those	sound	more	appealing	.	can	i	book	you	a	room	?	"
    #query = "I work with Apple Inc."

    f = TokenClassifier("./output")
    token_predict, logits = f.predict(query)
    print(query)
    print(token_predict)


In [ ]:
#@title CONLLEVAL_PERL (OFFLINE SCORING)
#offline scoring script
%%writefile conlleval_perl.py
#!/usr/bin/python
#### Original Perl Script
# conlleval: evaluate result of processing CoNLL-2000 shared task
# usage:     conlleval [-l] [-r] [-d delimiterTag] [-o oTag] < file
#            README: http://cnts.uia.ac.be/conll2000/chunking/output.html
# options:   l: generate LaTeX output for tables like in
#               http://cnts.uia.ac.be/conll2003/ner/example.tex
#            r: accept raw result tags (without B- and I- prefix;
#                                       assumes one word per chunk)
#            d: alternative delimiter tag (default is white space or tab)
#            o: alternative outside tag (default is O)
# note:      the file should contain lines with items separated
#            by $delimiter characters (default space). The final
#            two items should contain the correct tag and the
#            guessed tag in that order. Sentences should be
#            separated from each other by empty lines or lines
#            with $boundary fields (default -X-).
# url:       http://lcg-www.uia.ac.be/conll2000/chunking/
# started:   1998-09-25
# version:   2004-01-26
# author:    Erik Tjong Kim Sang <erikt@uia.ua.ac.be>
#### Now in Python
# author:    sighsmile.github.io
# version:   2017-05-18

from __future__ import division, print_function, unicode_literals
import argparse
import sys
from collections import defaultdict

# sanity check
def parse_args():
    argparser = argparse.ArgumentParser()
    argparser.add_argument(
        "-l", "--latex",
        default=False, action="store_true",
        help="generate LaTeX output"
    )
    argparser.add_argument(
        "-r", "--raw",
        default=False, action="store_true",
        help="accept raw result tags"
    )
    argparser.add_argument(
        "-d", "--delimiter",
        default=None,
        help="alternative delimiter tag (default: single space)"
    )
    argparser.add_argument(
        "-o", "--oTag",
        default="O",
        help="alternative delimiter tag (default: O)"
    )
    args = argparser.parse_args()
    return args



"""
• IOB1: I is a token inside a chunk, O is a token outside a chunk and B is the
beginning of chunk immediately following another chunk of the same Named Entity.
• IOB2: It is same as IOB1, except that a B tag is given for every token, which exists at
the beginning of the chunk.
• IOE1: An E tag used to mark the last token of a chunk immediately preceding another
chunk of the same named entity.
• IOE2: It is same as IOE1, except that an E tag is given for every token, which exists at
the end of the chunk.
• START/END: This consists of the tags B, E, I, S or O where S is used to represent a
chunk containing a single token. Chunks of length greater than or equal to two always
start with the B tag and end with the E tag.
• IO: Here, only the I and O labels are used. This therefore cannot distinguish between
adjacent chunks of the same named entity.

"""
# endOfChunk: checks if a chunk ended between the previous and current word
# arguments:  previous and current chunk tags, previous and current types
# note:       this code is capable of handling other chunk representations
#             than the default CoNLL-2000 ones, see EACL'99 paper of Tjong
#             Kim Sang and Veenstra http://xxx.lanl.gov/abs/cs.CL/9907006
def endOfChunk(prevTag, tag, prevType, type):
    """
    checks if a chunk ended between the previous and current word;
    arguments:  previous and current chunk tags, previous and current types
    """
    return ((prevTag == "B" and tag == "B") or
        (prevTag == "B" and tag == "O") or
        (prevTag == "I" and tag == "B") or
        (prevTag == "I" and tag == "O") or

        (prevTag == "E" and tag == "E") or
        (prevTag == "E" and tag == "I") or
        (prevTag == "E" and tag == "O") or
        (prevTag == "I" and tag == "O") or

        (prevTag != "O" and prevTag != "." and prevType != type) or
        (prevTag == "]" or prevTag == "["))
        # corrected 1998-12-22: these chunks are assumed to have length 1


# startOfChunk: checks if a chunk started between the previous and current word
# arguments:    previous and current chunk tags, previous and current types
# note:         this code is capable of handling other chunk representations
#               than the default CoNLL-2000 ones, see EACL'99 paper of Tjong
#               Kim Sang and Veenstra http://xxx.lanl.gov/abs/cs.CL/9907006
def startOfChunk(prevTag, tag, prevType, type):
    """
    checks if a chunk started between the previous and current word;
    arguments:  previous and current chunk tags, previous and current types
    """
    chunkStart = ((prevTag == "B" and tag == "B") or
        (prevTag == "B" and tag == "B") or
        (prevTag == "I" and tag == "B") or
        (prevTag == "O" and tag == "B") or
        (prevTag == "O" and tag == "I") or

        (prevTag == "E" and tag == "E") or
        (prevTag == "E" and tag == "I") or
        (prevTag == "O" and tag == "E") or
        (prevTag == "O" and tag == "I") or

        (tag != "O" and tag != "." and prevType != type) or
        (tag == "]" or tag == "["))
        # corrected 1998-12-22: these chunks are assumed to have length 1

    #print("startOfChunk?", prevTag, tag, prevType, type)
    #print(chunkStart)
    return chunkStart

def calcMetrics(TP, P, T, percent=True):
    """
    compute overall precision, recall and FB1 (default values are 0.0)
    if percent is True, return 100 * original decimal value
    """
    precision = TP / P if P else 0
    recall = TP / T if T else 0
    FB1 = 2 * precision * recall / (precision + recall) if precision + recall else 0
    if percent:
        return 100 * precision, 100 * recall, 100 * FB1
    else:
        return precision, recall, FB1

def splitTag(chunkTag, oTag = "O", raw = False):
    """
    Split chunk tag into IOB tag and chunk type;
    return (iob_tag, chunk_type)
    """
    if chunkTag == "O" or chunkTag == oTag:
        tag, type = "O", None
    elif raw:
        tag, type = "B", chunkTag
    else:
        try:
            # split on first hyphen, allowing hyphen in type
            tag, type = chunkTag.split('-', 1)
        except ValueError:
            tag, type  = chunkTag, None
    return tag, type

def countChunks(fileIterator, args):
    """
    Process input in given format and count chunks using the last two columns;
    return correctChunk, foundGuessed, foundCorrect, correctTags, tokenCounter
    """
    #boundary = "-X-"     # sentence boundary
    boundary = ""     # sentence boundary
    delimiter = args.delimiter
    raw = args.raw
    oTag = args.oTag

    correctChunk = defaultdict(int)     # number of correctly identified chunks
    foundCorrect = defaultdict(int)     # number of chunks in corpus per type
    foundGuessed = defaultdict(int)     # number of identified chunks per type

    tokenCounter = 0     # token counter (ignores sentence breaks)
    correctTags = 0      # number of correct chunk tags

    lastType = None # temporary storage for detecting duplicates
    inCorrect = False # currently processed chunk is correct until now
    lastCorrect, lastCorrectType = "O", None    # previous chunk tag in corpus
    lastGuessed, lastGuessedType = "O", None  # previously identified chunk tag

    for line in fileIterator:
        # each non-empty line must contain >= 3 columns
        features = line.strip().split(delimiter)
        if not features or features[0] == boundary:
            features = [boundary, "O", "O"]
        elif len(features) < 3:
             raise IOError("conlleval: unexpected number of features in line %s\n" % line)

        # extract tags from last 2 columns
        guessed, guessedType = splitTag(features[-1], oTag=oTag, raw=raw)
        correct, correctType = splitTag(features[-2], oTag=oTag, raw=raw)

        # 1999-06-26 sentence breaks should always be counted as out of chunk
        firstItem = features[0]
        if firstItem == boundary:
            guessed, guessedType = "O", None

        # decide whether current chunk is correct until now
        if inCorrect:
            endOfGuessed = endOfChunk(lastCorrect, correct, lastCorrectType, correctType)
            endOfCorrect = endOfChunk(lastGuessed, guessed, lastGuessedType, guessedType)
            if (endOfGuessed and endOfCorrect and lastGuessedType == lastCorrectType):
                inCorrect = False
                correctChunk[lastCorrectType] += 1
            elif ( endOfGuessed != endOfCorrect or guessedType != correctType):
                inCorrect = False

        startOfGuessed = startOfChunk(lastGuessed, guessed, lastGuessedType, guessedType)
        startOfCorrect = startOfChunk(lastCorrect, correct, lastCorrectType, correctType)
        if (startOfCorrect and startOfGuessed and guessedType == correctType):
            inCorrect = True
        if startOfCorrect:
            foundCorrect[correctType] += 1
        if startOfGuessed:
            foundGuessed[guessedType] += 1

        if firstItem != boundary:
            if correct == guessed and guessedType == correctType:
                correctTags += 1
            tokenCounter += 1

        lastGuessed, lastGuessedType = guessed, guessedType
        lastCorrect, lastCorrectType = correct, correctType

    if inCorrect:
        correctChunk[lastCorrectType] += 1

    return correctChunk, foundGuessed, foundCorrect, correctTags, tokenCounter

def evaluate(correctChunk, foundGuessed, foundCorrect, correctTags, tokenCounter, latex=False):
    # sum counts
    correctChunkSum = sum(correctChunk.values())
    foundGuessedSum = sum(foundGuessed.values())
    foundCorrectSum = sum(foundCorrect.values())

    # sort chunk type names
    sortedTypes = list(foundCorrect) + list(foundGuessed)
    sortedTypes = list(set(sortedTypes))
    sortedTypes.sort()

    # print overall performance, and performance per chunk type
    if not latex:
        # compute overall precision, recall and FB1 (default values are 0.0)
        precision, recall, FB1 = calcMetrics(correctChunkSum, foundGuessedSum, foundCorrectSum)
        # print overall performance
        print("processed %i tokens with %i phrases; " % (tokenCounter, foundCorrectSum), end='')
        print("found: %i phrases; correct: %i.\n" % (foundGuessedSum, correctChunkSum), end='')
        if tokenCounter:
            print("accuracy: %6.2f%%; " % (100*correctTags/tokenCounter), end='')
            print("precision: %6.2f%%; recall: %6.2f%%; FB1: %6.2f" %
                    (precision, recall, FB1))

        for i in sortedTypes:
            precision, recall, FB1 = calcMetrics(correctChunk[i], foundGuessed[i], foundCorrect[i])
            print("%17s: " %i , end='')
            print("precision: %6.2f%%; recall: %6.2f%%; FB1: %6.2f" %
                    (precision, recall, FB1), end='')
            print("  %d" % foundGuessed[i])

    # generate LaTeX output for tables like in
    # http://cnts.uia.ac.be/conll2003/ner/example.tex
    else:
        print("        & Precision &  Recall  & F\$_{\\beta=1} \\\\\\hline", end='')
        for i in sortedTypes:
            precision, recall, FB1 = calcMetrics(correctChunk[i], foundGuessed[i], foundCorrect[i])
            print("\n%-7s &  %6.2f\\%% & %6.2f\\%% & %6.2f \\\\" %
                 (i,precision,recall,FB1), end='')
        print("\\hline")

        precision, recall, FB1 = calcMetrics(correctChunkSum, foundGuessedSum, foundCorrectSum)
        print("Overall &  %6.2f\\%% & %6.2f\\%% & %6.2f \\\\\\hline" %
              (precision,recall,FB1))

if __name__ == "__main__":
    args = parse_args()
    # process input and count chunks
    correctChunk, foundGuessed, foundCorrect, correctTags, tokenCounter = countChunks(sys.stdin, args)

    # compute metrics and print
    evaluate(correctChunk, foundGuessed, foundCorrect, correctTags, tokenCounter, latex=args.latex)

    sys.exit(0)

In [ ]:
#offline scoring with conlleval_perl.py (will raise error if the 2 files have different length or format)
! awk -F"\t" '{print $NF}' M1test_predictions_ne_other.txt | paste test_ne_other.txt - | python3 conlleval_perl.py 

In [ ]:
#run this to write results to correct directory
%cd /content/drive/MyDrive/Research/ner/model_results

/content/drive/MyDrive/Research/ner/model_results


In [ ]:
#writes file to model_results, must run code above
%%writefile M1test_o_predres.txt
! awk -F"\t" '{print $NF}' M1test_predictions_o.txt | paste test_o.txt - | python3 conlleval_perl.py 
processed 259765 tokens with 0 phrases; found: 17 phrases; correct: 0.
accuracy:  99.98%; precision:   0.00%; recall:   0.00%; FB1:   0.00
            hotel: precision:   0.00%; recall:   0.00%; FB1:   0.00  4
       restaurant: precision:   0.00%; recall:   0.00%; FB1:   0.00  13

Overwriting M1test_o_predres.txt


In [ ]:
#return to ner
%cd ..

/content/drive/MyDrive/Research/ner


In [ ]:
#@title MANUAL DATA TUNING
#output csv file of NE entities and their occurances, for data tuning
ptrain=open('M2train_predictions_o.txt','r')
output=open('DATATUNE.csv','w')
dic={}
sen=[]
lab=[]
for line in ptrain:
    if line!='\n':
        elemlst=line.split('\t')
        sen.append(elemlst[0])
        lab.append(elemlst[1])
    elif line=='\n':
        for i in range(len(lab)):
            if lab[i][0]=='B':
                count=1
                while i+count<=len(lab)-1:
                    if lab[i+count][0]=='I':
                        count+=1
                    else:
                        break
                if count==2:
                    if i+count>=len(lab):
                        tokens=''
                        for j in range(i,i+count):
                            if sen[j]!=',':
                              tokens+=sen[j].strip()
                            if j!=i+count-1:
                                tokens+='_'
                        new=tokens.strip()
                        if new in dic:
                            dic[new][1]+=1
                        else:
                            dic[new]=[lab[i],1]
                    else:
                        tokens=''
                        for j in range(i,i+count+1):
                            if sen[j]!=',':
                                tokens+=sen[j].strip()
                            if j!=i+count:
                                tokens+='_'
                        new=tokens.strip()
                        if new in dic:
                            dic[new][1]+=1
                        else:
                            dic[new]=[lab[i],1]
                elif count>=3:
                    tokens=''
                    for j in range(i,i+count):
                        if sen[j]!=',':
                            tokens+=sen[j].strip()
                        if j!=i+count-1:
                            tokens+='_'
                    new=tokens.strip()
                    if new in dic:
                        dic[new][1]+=1
                    else:
                        dic[new]=[(lab[i]),1]
        sen=[]
        lab=[]
for key in dic:
    if key!='\n':
        output.write(key.strip()+','+str(dic[key][0]).strip()+','+str(dic[key][1]).strip()+'\n')

ptrain.close()
output.close()


In [ ]:
#starting elasticsearch sever
%cd /content
#!cp -r ~/Research/ner/elasticsearch-7.0.0 /content
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0
#!chmod -R 777 elasticsearch-7.0.0/
!ls -l | grep elastic
#!chmod -R 777 elasticsearch-7.0.0/bin/elasticsearch
#!ls -l elasticsearch-7.0.0/bin/elasticsearch
#!rm -rf elasticsearch-7.0.0

/content
drwxr-xr-x 10 daemon daemon      4096 Apr  5  2019 elasticsearch-7.0.0
-rw-r--r--  1 root   root   346760090 Apr 10  2019 elasticsearch-7.0.0-linux-x86_64.tar.gz
-rw-r--r--  1 root   root   346760090 Apr 10  2019 elasticsearch-7.0.0-linux-x86_64.tar.gz.1


In [ ]:
#checks for accessability
'''
should come out to be:
/content
total 12
drwx------ 6 root   root   4096 Jan 21 02:50 drive
drwx------ 9 daemon daemon 4096 Jan 21 04:10 elasticsearch-7.0.0
drwxr-xr-x 1 root   root   4096 Jan 15 17:28 sample_data
'''
!pwd
!ls -l 
#%cd MyDrive/Research/ner

/content
total 677292
drwx------  6 root   root        4096 Jan 22 03:11 drive
drwxr-xr-x 10 daemon daemon      4096 Apr  5  2019 elasticsearch-7.0.0
-rw-r--r--  1 root   root   346760090 Apr 10  2019 elasticsearch-7.0.0-linux-x86_64.tar.gz
-rw-r--r--  1 root   root   346760090 Apr 10  2019 elasticsearch-7.0.0-linux-x86_64.tar.gz.1
-rw-r--r--  1 root   root           0 Jan 22 07:09 elasticsearch_results.txt
drwxr-xr-x  1 root   root        4096 Jan 20 17:27 sample_data


In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

In [ ]:
#wait a bit to start the server
!curl -X GET "localhost:9200/"

{
  "name" : "741f4f8fb092",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "ugb_m-K5TYGPDTOOzilxCw",
  "version" : {
    "number" : "7.0.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "b7e28a7",
    "build_date" : "2019-04-05T22:55:32.697037Z",
    "build_snapshot" : false,
    "lucene_version" : "8.0.0",
    "minimum_wire_compatibility_version" : "6.7.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
!pip install elasticsearch==7.7.1

In [ ]:
import elasticsearch
print(elasticsearch.__path__)
!ls -l /usr/local/lib/python3.6/dist-packages/elasticsearch/__init__.py

In [ ]:
from datasets import load_dataset
squad = load_dataset('squad', split='validation')
squad.add_elasticsearch_index("context", host="localhost", port="9200")

In [ ]:
squad.get_index("context").es_index_name

'huggingface_datasets_tmp0cfwk0bt'

In [ ]:
#testing with squad dataset on elasticsearch
query = "machine"
scores, retrieved_examples = squad.get_nearest_examples("context", query)
retrieved_examples,scores

In [ ]:
#@title KNOWLEDGE BASE TRAIN DATASCRIPT
%%writefile knowledge_base_train.py
import logging
import json
import datasets


_CITATION = ''

_DESCRIPTION = ''
_URL = ""
#_TRAINING_FILE = 'train_ne.txt'
#_DEV_FILE = 'valid_ne.txt'
#_TEST_FILE = 'train_o.txt'
_KNOWLEDGE_FILE = 'knowledge.json'

class knowledge_base_trainConfig(datasets.BuilderConfig):
    """BuilderConfig for knowledge_base_train"""

    def __init__(self, **kwargs):
        """BuilderConfig for knowledge_base_train.
        Args:
          **kwargs: keyword arguments forwarded to super.
        """
        super(knowledge_base_trainConfig, self).__init__(**kwargs)


class knowledge_base_train(datasets.GeneratorBasedBuilder):
    """knowledge_base_train dataset."""

    BUILDER_CONFIGS = [
        knowledge_base_trainConfig(name="knowledge_base_train", version=datasets.Version("1.0.0"), description="knowledge base for training logs (under simple_ner2021)"),
    ]

    def _info(self):
        return datasets.DatasetInfo(
            description=_DESCRIPTION,
            features=datasets.Features(
                {
                    "id": datasets.Value("string"),
                    "tokens": datasets.Value("string"),
                    "category": datasets.Value("string"),
                    "title":datasets.Sequence(datasets.Value("string")),
                    "body":datasets.Sequence(datasets.Value("string")),
                }
            ),
            supervised_keys=None,
            homepage="N/A",
            citation=_CITATION,
        )
    
    def _split_generators(self, dl_manager):
        """Returns SplitGenerators."""
        urls_to_download = {
            "knowledge": f"{_URL}{_KNOWLEDGE_FILE}",
        }
        downloaded_files = dl_manager.download_and_extract(urls_to_download)

        return [
            datasets.SplitGenerator(name=datasets.Split.TRAIN, gen_kwargs={"filepath": downloaded_files["knowledge"]}),
        ]
    
    def _generate_examples(self, filepath):
        logging.info("⏳ Generating examples from = %s", filepath)
        with open(filepath, encoding="utf-8") as f:
            dic=json.load(f)
            guid = 0
            for cat in dic:
                for elem in dic[cat]:
                    entity=dic[cat][elem]['name']
                    if entity is not None:
                        entity=entity.strip()
                    for conversation in dic[cat][elem]['docs']:
                        title.append(dic[cat][elem]['docs'][conversation]['title'].strip())
                        body.append(dic[cat][elem]['docs'][conversation]['body'].strip())
                    yield guid,{
                        "id":elem,
                        "tokens":entity,
                        "category":cat,
                        "title":title,
                        "body":body,
                    }
                    guid+=1
                    title=[]
                    body=[]

In [ ]:
%cd MyDrive/Research/ner

/content/drive/MyDrive/Research/ner


In [ ]:
#loading knowledge base to elasticsearch
from datasets import load_dataset
kbase = load_dataset('knowledge_base_train.py', split='train')
kbase.add_elasticsearch_index("tokens", host="localhost", port="9200")

Dataset knowledge_base_train downloaded and prepared to /root/.cache/huggingface/datasets/knowledge_base_train/knowledge_base_train/1.0.0/2f8298e9f6fea43fd87af44de70525a9d04cd4e4aa010b6028c9485bf5a97eed. Subsequent calls will reuse this data.


Dataset({
    features: ['id', 'tokens', 'category', 'title', 'body'],
    num_rows: 145
})

In [ ]:
query = 'Ali Baba'
try:
    scores, retrieved_examples = kbase.get_nearest_examples('tokens',query,k=10)
    print(retrieved_examples,scores)
except:
    print('failed')

In [ ]:
kbase.get_index("tokens").es_index_name

'huggingface_datasets_tmpydre60h3'

In [ ]:
kbase['tokens'][42]

'NANDOS'

In [ ]:
%cd /content

/content


In [ ]:
import time
from datasets.search import SearchResults
from elasticsearch import Elasticsearch

def es_search(es_client,es_index_name, query: str) -> SearchResults:
    response = es_client.search(
        index=es_index_name,
        body={
  "query": {
    "match": {
      "text": {
        "query": query,
        "fuzziness": 1,
        "prefix_length": 0,
      }
    }
  },
  "highlight":{
      "order":'score',
      "fields":{
          "text":{
              "type":"plain"
              }
          }
      }
      }
    )
    hits = response["hits"]["hits"]
    highlight_lst=[]
    for hit in hits:
        if 'highlight' in hit:
            highlight_lst.append(hit['highlight']['text'])
        time.sleep(1)
    return SearchResults([hit["_score"] for hit in hits], [int(hit["_id"]) for hit in hits]),highlight_lst

def main():
    es_client = Elasticsearch([{"host": "localhost", "port": "9200"}]) 
    es_index_name='huggingface_datasets_tmpydre60h3'
    examples, highlights = es_search(es_client,es_index_name,'Looking for a place to eat in the nandos')
    print(examples[0],examples[1],highlights,sep='\n')
    
    
if __name__ == "__main__":
    main()

#modify config: must{'term':{}}
#ordering + distance window<=1 (slop)
#coverage of each result, coverage first then score

[5.3802414, 3.9867263, 3.758799, 2.9774845, 2.4695122, 2.3011153, 2.3011153, 2.3011153, 2.3011153, 1.954372]
[42, 130, 41, 128, 0, 87, 106, 122, 127, 28]
[['<em>NANDOS</em>'], ['TANDOORI <em>PALACE</em>'], ['<em>NANDOS</em> CITY CENTRE'], ['RESTAURANT <em>TWO</em> <em>TWO</em>'], ['<em>A</em> AND B GUEST HOUSE'], ['<em>THE</em> GANDHI'], ['<em>THE</em> GARDENIA'], ['<em>THE</em> HOTPOT'], ['<em>THE</em> NIRALA'], ['<em>THE</em> CAMBRIDGE BELFRY']]


# Task 2: Incorporate dictionary features into the model

* Use Trie to locate the positions of each entity in a sentence.
* Feed in positions as extra features into the model.
* During test time, we can add new entities from knowledge base into the dictionary to improve model robustness on recognizing the new named entities not covered during training.

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 7.2MB/s 
     |████████████████████████████████| 901kB 37.4MB/s 
     |████████████████████████████████| 3.3MB 47.3MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoConfig

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
config = AutoConfig.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

In [ ]:
outputs = tokenizer("I work with Apple Inc.")
print(outputs.input_ids)

[101, 1045, 2147, 2007, 6207, 4297, 1012, 102]


In [ ]:
tokenizer.batch_decode(outputs.input_ids)

['[CLS]', 'i', 'work', 'with', 'apple', 'inc', '.', '[SEP]']

In [ ]:
print(config)

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.5.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

